In [ ]:
import math
import random
import re
from datetime import datetime, timedelta
from itertools import combinations

import numpy as np
import pandas as pd
import spacy
import torch
from pulp import LpMinimize, LpProblem, LpVariable, lpSum
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModel, AutoTokenizer, pipeline

# 1. Анализ данных и их обработка

## 1.1 Список поручений

In [68]:
df_cases = pd.read_csv("../data/cases.csv", delimiter=";", encoding="utf-8-sig", decimal=",")
df_cases.head(5)

,Номер поручения,Заказчик,Дата поручения,Выполнено,Дата выполнения,Затрачено дней,Сумма вознаграждения,Описание
0,11000,Анна,1051-08-11,да,1051-08-21,4.0,6000,В лесу по дороге от пещеры звери нападают на л...
1,11001,Мария,1051-07-09,да,1051-09-02,2.0,20000,В лесу по дороге от пещеры были замечены разбо...
2,11002,Эмилио,1053-10-05,да,1053-10-20,7.0,22500,По дороге из деревни монстры похитили путников...
3,11003,Бьянка,1052-11-24,да,1052-12-02,5.0,5500,Недалеко от города видели монстров. Нужно побе...
4,11004,Бьянка,1052-02-23,да,1052-03-30,8.0,10500,В деревне у меня пропала сумка с документами. ...


На основе первичного анализа можно выделить несколько ключевых особенностей:

1. **Повторяющийся характер описаний**  
   В описаниях поручений наблюдаются схожие фразы и структуры, что указывает на их стандартный характер.

2. **Явная структура текста**  
   Каждое описание состоит из двух или трёх предложений, которые соответствуют следующей логике:
   - **Описание ситуации:** Указание на проблему или событие, произошедшее в определённой местности (например, нападение монстров или кража).
   - **Необходимая цель:** Формулировка задачи, которую требуется выполнить (например, победить монстров или вернуть пропавший предмет).
   - **Совет:** В некоторых случаях добавляется дополнительное указание или пожелание, уточняющее действия или способы выполнения задания.

Такой подход позволяет быстро понять суть поручения и его требования.


In [69]:
df_cases["Описание"].value_counts()

Описание
В пещере появвилось огромное каменное чудовище. Нужно убить его.                                                              23
В пещере появвилось огромное каменное чудовище. Нужно уничтожить его.                                                         22
В пещере завёлся дракон. Нужно его уничтожить.                                                                                19
В пещере завёлся дракон. Нужно его убить.                                                                                     18
В пещере завёлся дракон. Нужно его уничтожить. Это может потребовать времени и усилий, так как дракон очень опасен.           15
                                                                                                                              ..
В лесу между городом и деревней заметили зверей. Нужно прогнать их.                                                            1
В лесу по дороге от пещеры видели монстров. Нужно победить их.                          


На основе выявленных первичных зависимостей разработан метод, который разбивает описание на логические структуры:

In [70]:
#Разбиение описания на предложения и получение отдельного дата фрейма: номер поручения, описание, цель, совет
def split_description(data):
    ids = []
    description = []
    purpose = []
    advice = []
    for id, text in data:
        text_splitted = text.split(". ")
        ids.append(id)
        if len(text_splitted) == 2:
            description.append(text_splitted[0])
            purpose.append(text_splitted[1])
            advice.append("")
        else:
            description.append(text_splitted[0])
            purpose.append(text_splitted[1])
            advice.append(text_splitted[2])
    return pd.DataFrame({
        "Номер поручения": ids,
        "description": description,
        "advice": advice,
        "purpose": purpose
    })

После чего полученное "разбитое" описание было добавлено в исходную таблицу поручений:

In [71]:
#Разбиение описания и расширение таблицы с поручениями
df_cases = pd.merge(
    df_cases,
    split_description(df_cases[["Номер поручения", "Описание"]].values),
    on="Номер поручения",
    how="outer"
    )

Рассмотрим частоту встречаемости целей

In [72]:
df_cases["purpose"].value_counts()

purpose
Нужно найти её как можно скорее.       36
Нужно вернуть её как можно скорее.     35
Нужно убить их.                        34
Нужно прогнать их.                     27
Нужно спасти их                        23
Нужно убить его.                       23
Нужно вернуть его как можно скорее.    23
Нужно уничтожить его.                  22
Нужно освободить их                    21
Нужно найти его как можно скорее.      21
Нужно вернуть её как можно скорее      20
Нужно его уничтожить.                  19
Нужно уничтожить их.                   19
Нужно победить их.                     18
Нужно его убить.                       18
Нужно прогнать их                      16
Нужно его уничтожить                   15
Нужно убить их                         14
Нужно проучить их.                     14
Нужно найти её как можно скорее        13
Нужно освободить их.                   13
Нужно уничтожить их                    10
Нужно спасти их.                       10
Нужно его убить           

На основе предоставленных данных можно сделать следующие выводы:

1. **Схожесть целей задач**  
   Большинство задач имеют повторяющиеся или схожие формулировки целей, что позволяет выделить их в общие кластеры.

2. **Явные кластеры задач**  
   Цели можно разделить на несколько основных категорий:
   - **Поиск:** Например, "Нужно найти её как можно скорее" и "Нужно найти его как можно скорее".
   - **Возврат:** Например, "Нужно вернуть её как можно скорее" и "Нужно вернуть его как можно скорее".
   - **Уничтожение:** Например, "Нужно убить их", "Нужно уничтожить его" и "Нужно победить их".
   - **Освобождение:** Например, "Нужно освободить их" и "Нужно спасти их".
   - **Прогон:** Например, "Нужно прогнать их" и "Нужно проучить их".

3. **Разные формулировки одной цели**  
   Некоторые цели написаны разными словами, но представляют собой одно и то же действие. Например:
   - "Нужно уничтожить их" и "Нужно их уничтожить".
   - "Нужно убить его" и "Нужно его убить".

Такое дублирование можно оптимизировать, унифицировав формулировки задач для удобства анализа и обработки.


Для унификации описаний целей задач применены методы нормализации и лемматизации текста. Эти подходы позволяют эффективно обработать текстовые данные, устранив избыточность и разнообразие формулировок.  

## Что такое нормализация и лемматизация?

1. **Нормализация**  
   Нормализация текста включает удаление избыточных элементов, таких как:
   - **Стоп-слова** (например, предлоги, союзы), которые не несут ключевой информации.
   - **Знаки пунктуации**, которые не влияют на смысл текста.  
   Это позволяет сосредоточиться исключительно на значимых словах и фразах.

2. **Лемматизация**  
   Лемматизация приводит слова к их базовой форме (лемме). Например:
   - "убить", "убийца" → "убить".
   - "найти", "нашёл" → "найти".  
   Таким образом, все варианты написания и склонения одного слова сводятся к его основному виду, что упрощает дальнейшую обработку текста.

## Как это реализовано?

Для выполнения нормализации и лемматизации был разработан специальный класс на Python. Он использует библиотеку SpaCy, которая предоставляет инструменты для анализа текста на естественном языке. Основной метод класса, `normalize_and_lemmatize`, выполняет следующие шаги:

1. Загружает предобученную модель для русского языка `ru_core_news_sm`.
2. Обрабатывает текст, выделяя леммы слов.
3. Исключает из текста стоп-слова и знаки пунктуации.
4. Возвращает нормализованный текст в единой унифицированной форме.

In [73]:
class TextCasePreprocessing():
    def __init__(self) -> None:
        self.nlp_model = spacy.load("ru_core_news_sm")

    def normalize_and_lemmatize(self, text):
        doc = self.nlp_model(text)
        normalized_text = " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])
        return normalized_text


Преобразуем цели поручений на основе выше описанных методов:

In [74]:
text_preprocessor = TextCasePreprocessing()

In [75]:
df_cases["purpose_prep"] = df_cases["purpose"].apply(lambda x: text_preprocessor.normalize_and_lemmatize(x))
df_cases["description_prep"] = df_cases["description"].apply(lambda x: text_preprocessor.normalize_and_lemmatize(x))


На основе нормализованных данных целей задач можно выделить основные кластеры, представленные в таблице ниже:
## Первичный анализ

1. **Явные кластеры**  
   Каждый кластер представляет определённую цель задачи, например, уничтожение врагов, поиск предметов или спасение людей. Без учёта синонимов данные показывают разнообразие формулировок, которые уже нормализованы.

2. **Потенциальные объединения**  
   При детальном анализе можно заметить, что некоторые кластеры представляют схожие или даже идентичные задачи:
   - **"Убить" и "Уничтожить":** Оба действия указывают на необходимость устранения угрозы.
   - **"Освободить" и "Спасти":** В обоих случаях цель состоит в помощи или освобождении жертв.
   - **"Найти" и "Вернуть":** Задачи могут подразумевать поиск и возвращение одного и того же объекта.

3. **Рекомендации по объединению**  
   Для оптимизации анализа и дальнейшей обработки данных можно объединить некоторые кластеры:
   - Кластер **устранения угроз:** "Убить", "Уничтожить", "Победить".
   - Кластер **помощи:** "Спасти", "Освободить".
   - Кластер **поиска и возврата:** "Найти", "Вернуть".

## Вывод

Объединение схожих кластеров позволит более точно классифицировать задачи и выявить основные направления работы. Такой подход значительно упростит дальнейшую работу с данными, обеспечив их структурированность и уменьшив избыточность.  

In [76]:
df_cases["purpose_prep"].value_counts()

purpose_prep
убить         101
уничтожить     89
вернуть        82
найти          74
прогнать       43
освободить     34
спасти         33
победить       25
проучить       19
Name: count, dtype: int64

Рассмотрим предлагаемые кластеры отдельно и проведём их визуальную оценку на предмет близости формулировок и целей. Такой подход поможет понять, насколько предложенные объединения логически оправданы и соответствуют смыслу задач.

## Предлагаемые кластеры

1. **Кластер устранения угроз**  
   Включает цели:
   - "Убить"
   - "Уничтожить"
   - "Победить"  
   Все формулировки объединяет одна цель — устранение противника или угрозы. Однако стоит проверить, не содержат ли данные описания специфических нюансов, которые могут повлиять на точность объединения.

2. **Кластер помощи**  
   Включает цели:
   - "Спасти"
   - "Освободить"  
   Оба термина подразумевают помощь или защиту. Визуальная оценка позволит определить, есть ли различия в контексте задач (например, "освободить из плена" и "спасти от монстров").

3. **Кластер поиска и возврата**  
   Включает цели:
   - "Найти"
   - "Вернуть"  
   Здесь объединение кажется очевидным, так как поиск часто является частью задачи по возврату. Однако нужно проверить, встречаются ли исключительные случаи, где контекст существенно отличается.

4. **Дополнительные кластеры**  
   Такие цели, как "Прогнать" и "Проучить", формируют менее крупные, но всё же значимые группы. Следует оценить их контекст на предмет возможного объединения или уточнения.

## Подход к оценке

Для каждой пары целей внутри кластера:
- Проведём визуальное сравнение описаний задач, чтобы определить степень их сходства.
- Проверим, не содержат ли задачи контекстуальные различия, которые могут затруднить объединение.
- Убедимся, что объединение не приведёт к потере важной информации.

**Цель:** Обеспечить, чтобы предложенные кластеры были максимально точными и отражали истинные смыслы формулировок, заложенных в задачах.

In [ ]:
#Метод отображения описаний поручений на основе их целей
def show_cases_by_purposes(purpose_list):
    return df_cases[df_cases["purpose_prep"].isin(purpose_list)][["description_prep", "purpose_prep"]]

## Убить-Уничтожить-Победить"

На основании анализа таблицы можно сделать вывод, что задачи, связанные с уничтожением угроз, имеют общую цель, несмотря на разнообразие формулировок. Рассмотрим ключевые особенности этого кластера.

## Общее между задачами
1. **Единая цель:**  
   Задачи из этого кластера сводятся к устранению угроз, будь то монстры, разбойники или звери. Например:
   - "Пещера завелся дракон" требует действия "убить" или "уничтожить".
   - "Лес, город, деревня, монстры нападают на человека" может подразумевать как "уничтожить", так и "победить".

2. **Контекст угрозы:**  
   Независимо от используемой формулировки, описание задачи указывает на необходимость противодействия активной опасности (нападение монстров, разбойников или животных).

3. **Вариативность формулировок:**  
   - "Уничтожить" чаще используется для описания задач, связанных с монстрами или крупными существами ("каменное чудовище", "дракон").
   - "Убить" встречается в задачах, связанных с людьми (разбойники) или мелкими угрозами (звери).
   - "Победить" используется для обозначения более обобщённого результата, подразумевая победу над угрозой без уточнения её формы.

## Почему кластер работает

- **Контекстная близость:**  
  Независимо от точной формулировки ("убить", "уничтожить", "победить"), цель остаётся общей — устранение угрозы. Например:
  - "Лес, город, заметить монстра" может предполагать любое из трёх действий, в зависимости от предпочтений автора задачи.

- **Универсальность объединения:**  
  Объединение этих задач в один кластер позволяет эффективно анализировать их без потери значимого контекста.

## Вывод
Кластер "Убить-Уничтожить-Победить" обладает высокой степенью логической связанности. Формулировки "убить", "уничтожить" и "победить" могут быть использованы как взаимозаменяемые для описания одной и той же цели — ликвидации угрозы. Такое объединение оправдано и не вызывает потери важной информации.


In [78]:
show_cases_by_purposes(["убить", "победить", "уничтожить"]).value_counts()

description_prep                              purpose_prep
пещера завестись дракон                       уничтожить      34
пещера появвилось огромный каменный чудовище  убить           27
                                              уничтожить      26
пещера завестись дракон                       убить           26
лес город заметить разбойник                  убить            5
лес город деревня видеть монстр               победить         4
лес дорога пещера заметить монстр             победить         4
лес дорога пещера видеть монстр               уничтожить       4
лес деревня видеть монстр                     уничтожить       4
лес город деревня монстр нападать человек     уничтожить       4
город разбойник нападать человек              убить            3
лес город видеть монстр                       уничтожить       3
город встречаться звери                       убить            3
лес город деревня заметить разбойник          убить            3
лес деревня встречаться звери  

# "Спасти-Освободить"

Кластер "Спасти-Освободить" объединяет задачи, связанные с оказанием помощи жертвам, которых похитили монстры. Несмотря на различия в формулировках, задачи имеют общий смысл и цель. Рассмотрим ключевые особенности этого кластера.

## Общее между задачами

1. **Единая цель:**  
   Задачи из кластера сводятся к освобождению похищенных путников из плена монстров. Например:
   - "Город, монстр похитил путников" подразумевает либо "спасти", либо "освободить" жертв.
   - "Дорога, деревня, монстр похитил путников" также требует действий, направленных на их возвращение.

2. **Контекст ситуации:**  
   Независимо от используемой формулировки, задачи описывают однотипный сценарий:
   - Монстры похищают путников.
   - Основная цель — вернуть жертв обратно.

3. **Вариативность формулировок:**  
   - "Спасти" используется чаще для эмоционально окрашенных описаний, подчеркивающих опасность для жертв.
   - "Освободить" акцентирует внимание на физическом устранении преград или пленения, что часто встречается в описаниях вроде "освободить путников из деревни".

## Почему кластер работает

- **Контекстная близость:**  
  Обе формулировки ("спасти" и "освободить") подразумевают действия по устранению угрозы и возвращению похищенных. Например:
  - "Город, монстр похитил путников" может подразумевать как "спасти", так и "освободить", в зависимости от акцента на эмоциональной или практической стороне задачи.

- **Взаимозаменяемость формулировок:**  
  Задачи с этими формулировками можно объединить в один кластер без потери смысла, так как обе цели представляют одну и ту же конечную задачу — помощь жертвам.

## Вывод

Кластер "Спасти-Освободить" логически обоснован. Независимо от использования слов "спасти" или "освободить", задача остаётся одной и той же: устранить угрозу и помочь похищенным путникам. Объединение этих задач в общий кластер позволяет упорядочить данные и избежать дублирования.


In [79]:
show_cases_by_purposes(["спасти", "освободить"]).value_counts()

description_prep                       purpose_prep
город монстр похитить путник           спасти          15
дорога деревня монстр похитить путник  освободить      13
город монстр похитить путник           освободить      12
деревня монстр похитить путник         спасти          10
                                       освободить       9
дорога деревня монстр похитить путник  спасти           8
Name: count, dtype: int64

# "Найти-Вернуть"

Кластер "Найти-Вернуть" объединяет задачи, связанные с поиском и возвратом утерянных или украденных вещей. Несмотря на различия в формулировках, все задачи имеют схожую конечную цель: обнаружить пропавший предмет и вернуть его владельцу. Рассмотрим ключевые особенности этого кластера.

## Общее между задачами

1. **Единая цель:**  
   Все задачи направлены на поиск или возврат пропавших вещей, таких как:
   - "Сумка с документами"
   - "Рюкзак"
   - "Драгоценность"

2. **Контекст ситуации:**  
   Сценарии, описанные в задачах, варьируются, но их суть остаётся неизменной:
   - **Пропажа:** Предмет был потерян или забыт.
   - **Кража:** Предмет был похищен, и его необходимо вернуть.

3. **Вариативность формулировок:**  
   - "Найти" чаще используется, когда акцент делается на сам процесс обнаружения предмета.
   - "Вернуть" подразумевает конечный результат, связанный с передачей предмета обратно владельцу.

## Почему кластер работает

- **Контекстная близость:**  
  Описание задач, как правило, указывает на одну и ту же последовательность действий:
  - Поиск предмета (найти).
  - Возврат владельцу (вернуть).  
  Например, "город, пропала сумка с документами" может предполагать как "найти", так и "вернуть", в зависимости от фокусировки на этапе задачи.

- **Логическая взаимосвязь:**  
  Задачи с формулировками "найти" и "вернуть" практически всегда связаны друг с другом, так как возврат предполагает предварительный поиск.

## Вывод

Кластер "Найти-Вернуть" является логически оправданным объединением. Несмотря на различия в акцентах (поиск или результат), обе формулировки представляют собой этапы одной задачи: обнаружить пропавшую вещь и вернуть её владельцу. Объединение этих задач упрощает их анализ и устранение дублирования.


In [80]:
show_cases_by_purposes(["найти", "вернуть"]).value_counts()

description_prep                          purpose_prep
город пропасть драгоценность              вернуть         11
город пропасть рюкзак                     вернуть          7
деревня украсть сумка документ            вернуть          6
город потеряться сумка документ           вернуть          6
                                          найти            6
город пропасть рюкзак                     найти            6
город пропасть сумка документ             вернуть          6
город украсть драгоценность               найти            6
город украсть рюкзак                      вернуть          6
                                          найти            5
город потеряться рюкзак                   найти            5
город потеряться драгоценность            найти            5
город украсть сумка документ              найти            5
деревня украсть рюкзак                    вернуть          4
город украсть сумка документ              вернуть          4
город потеряться драгоценность

# "Прогнать-Проучить"

Кластер "Прогнать-Проучить" объединяет задачи, связанные с устранением угрозы, будь то разбойники или звери. Несмотря на различие в формулировках, все задачи подразумевают необходимость избавления от источника проблемы. Рассмотрим основные особенности этого кластера.

## Общее между задачами

1. **Единая цель:**  
   Все задачи сводятся к устранению присутствия или влияния нежелательных элементов:
   - Разбойники, замеченные в лесу, деревне или на дороге.
   - Звери, нападающие на людей или встречающиеся на пути.

2. **Контекст ситуации:**  
   Формулировки описывают разные степени воздействия:
   - **Прогнать:** Указание на удаление угрозы без акцента на суровости действия.
   - **Проучить:** Подразумевает наказание или применение силы с целью предупреждения повторных действий.

3. **Вариативность формулировок:**  
   - "Прогнать" чаще используется для описания задачи, где требуется лишь устранить проблему.
   - "Проучить" акцентирует внимание на наказании или преподнесении урока, особенно в отношении разбойников.

## Почему кластер работает

- **Контекстная близость:**  
  Независимо от выбранного термина, обе формулировки относятся к задачам устранения угрозы. Например:
  - "Лес, город, заметить разбойников" может подразумевать как "прогнать" (удалить угрозу), так и "проучить" (наказать).

- **Логическая взаимосвязь:**  
  В большинстве случаев проучивание автоматически включает удаление угрозы. Поэтому объединение задач "прогнать" и "проучить" обосновано.

## Вывод

Кластер "Прогнать-Проучить" логически оправдан, так как обе формулировки описывают действия, направленные на устранение угрозы. Различия между "прогнать" и "проучить" в основном связаны с уровнем воздействия, но конечная цель остаётся одной и той же. Объединение этих задач в общий кластер помогает упростить анализ и повысить точность обработки данных.


In [81]:
show_cases_by_purposes(["прогнать", "проучить"]).value_counts()

description_prep                              purpose_prep
лес город деревня заметить разбойник          прогнать        5
лес город заметить разбойник                  проучить        4
лес дорога пещера зверь нападать человек      прогнать        3
город заметить разбойник                      прогнать        3
лес дорога пещера заметить зверь              прогнать        3
лес дорога пещера встречаться звери           прогнать        3
лес деревня разбойник нападать человек        прогнать        2
лес город заметить разбойник                  прогнать        2
лес деревня заметить зверь                    прогнать        2
лес деревня заметить разбойник                проучить        2
лес город деревня заметить разбойник          проучить        2
лес деревня разбойник нападать человек        проучить        2
дорога деревня разбойник нападать человек     прогнать        2
дорога деревня заметить разбойник             проучить        2
дорога деревня встречаться звери             

### Формирование основных кластеров поручений

На основе анализа задач удалось выделить 4 основных кластера целей:

1. **Убить-Победить-Уничтожить**
2. **Спасти-Освободить**
3. **Вернуть-Найти**
4. **Прогнать-Проучить**

Эти кластеры объединили схожие задачи, выраженные разными словами, в логически связанные группы. 

Для работы с этими кластерами используется структура `synonym_cliques`:

```python
synonym_cliques = [
    "убить-победить-уничтожить",
    "спасти-освободить",
    "вернуть-найти",
    "прогнать-проучить"
]

На основе этих кластеров был разработан метод `get_synonym_dict`, который автоматически формирует словарь синонимов. Этот словарь устанавливает правила замены, позволяя нормализовать цели задач.

In [82]:
def get_synonym_dict(synonym_cliques):
    synonym_dict = {}

    for clique in synonym_cliques:
        for word in clique.split("-"):
            synonym_dict[word] = clique
    return synonym_dict

In [83]:
synonym_cliques = [
    "убить-победить-уничтожить",
    "спасти-освободить",
    "вернуть-найти",
    "прогнать-проучить"
]

In [84]:
synonym_dict = get_synonym_dict(synonym_cliques)
synonym_dict

{'убить': 'убить-победить-уничтожить',
 'победить': 'убить-победить-уничтожить',
 'уничтожить': 'убить-победить-уничтожить',
 'спасти': 'спасти-освободить',
 'освободить': 'спасти-освободить',
 'вернуть': 'вернуть-найти',
 'найти': 'вернуть-найти',
 'прогнать': 'прогнать-проучить',
 'проучить': 'прогнать-проучить'}

Замена названия цели на соотвествующий кластер

In [85]:
df_cases["common_purpose"] = df_cases["purpose_prep"].map(synonym_dict)

In [86]:
df_cases.head(4)

,Номер поручения,Заказчик,Дата поручения,Выполнено,Дата выполнения,Затрачено дней,Сумма вознаграждения,Описание,description,advice,purpose,purpose_prep,description_prep,common_purpose
0,11000,Анна,1051-08-11,да,1051-08-21,4.0,6000,В лесу по дороге от пещеры звери нападают на л...,В лесу по дороге от пещеры звери нападают на л...,"Осмотрите все возможные укрытия, чтобы найти з...",Нужно убить их,убить,лес дорога пещера зверь нападать человек,убить-победить-уничтожить
1,11001,Мария,1051-07-09,да,1051-09-02,2.0,20000,В лесу по дороге от пещеры были замечены разбо...,В лесу по дороге от пещеры были замечены разбо...,"Будьте осторожны, так как разбойники могут быт...",Нужно убить их,убить,лес дорога пещера заметить разбойник,убить-победить-уничтожить
2,11002,Эмилио,1053-10-05,да,1053-10-20,7.0,22500,По дороге из деревни монстры похитили путников...,По дороге из деревни монстры похитили путников,"Осмотрите все возможные укрытия, чтобы найти п...",Нужно освободить их,освободить,дорога деревня монстр похитить путник,спасти-освободить
3,11003,Бьянка,1052-11-24,да,1052-12-02,5.0,5500,Недалеко от города видели монстров. Нужно побе...,Недалеко от города видели монстров,"Будьте осторожны, так как монстры могут быть о...",Нужно победить их,победить,город видеть монстр,убить-победить-уничтожить



Анализ данных показывает, что задачи можно разделить на 4 основных кластера целей. Их частота встречаемости подтверждает ключевые направления поручений:

#### 1. **Убить-Победить-Уничтожить**  
Это наиболее распространённый кластер поручений. Он включает задачи, связанные с устранением угроз в виде монстров, разбойников или зверей.  
Такие поручения составляют 36% от общего числа, что подчеркивает важность борьбы с опасностью в окружающем мире.

#### 2. **Вернуть-Найти**  
Второй по популярности кластер. Задачи этой группы связаны с поиском и возвратом пропавших или украденных вещей.  
Доля этого кластера составляет 26%, что делает его важным направлением поручений.

#### 3. **Спасти-Освободить**  
Кластер охватывает задачи, направленные на спасение или освобождение похищенных путников.  
Частота встречаемости этих поручений составляет 11%, что отражает их значимость, хотя и меньшую, чем у первых двух кластеров.

#### 4. **Прогнать-Проучить**  
Наименее частый кластер, который включает задачи по устранению угрозы в виде разбойников или зверей, без полного их уничтожения.  
Его доля составляет 10% от общего числа поручений.

### Вывод
Частота встречаемости показывает, что наибольшее внимание уделяется устранению угроз и поиску пропавших вещей. Задачи спасения и проучивания встречаются реже, но их важность остаётся значительной. Это распределение подчёркивает ключевые направления поручений, которые могут служить основой для дальнейшей унификации и анализа.


In [87]:
df_cases["common_purpose"].value_counts()

common_purpose
убить-победить-уничтожить    215
вернуть-найти                156
спасти-освободить             67
прогнать-проучить             62
Name: count, dtype: int64

### Определение кластеров и необходимость детализации

Мы выявили явные кластеры-цели поручений, которые отражают основные направления задач. Однако, для полного анализа необходимо дополнительно определить **объект**, над которым или от которого нужно выполнить указанную цель.

#### Зачем это нужно?
Цели, такие как "убить", "вернуть" или "спасти", сами по себе не дают полной картины без уточнения:
- Кто или что является объектом? (например, "монстр", "рюкзак", "путник").
- Где находится этот объект? (например, "в лесу", "в городе", "на дороге").  

#### Следующий шаг
Определение объекта и его характеристик позволит:
- Уточнить описание поручений.
- Упростить их классификацию.
- Выявить дополнительные закономерности в данных.  

Таким образом, вместе с целями важно учитывать и объекты, чтобы построить более точную и детализированную модель анализа.


### Извлечение объекта из описания

Для определения объекта, над которым или от которого нужно выполнить цель, был использовали метод автоматического извлечения информации с помощью трансформера. Эта задача решается с использованием библиотеки **Hugging Face Transformers** и предобученной модели **ruselkomp/deep-pavlov-full**, настроенной на задачу "вопрос-ответ".

#### Как это работает?

1. **Что такое модель "вопрос-ответ"?**  
   Это алгоритм, который принимает текст (описание задачи) и вопрос, а затем возвращает наиболее вероятный ответ, найденный в тексте. Например:
   - Текст: "Лес, город, дракон похитил путников."
   - Вопрос: "Кто является объектом задачи?"  
   - Ответ: "дракон".

2. **Используемая модель**  
   Модель **ruselkomp/deep-pavlov-full** обучена на русском языке для задачи "вопрос-ответ". Она анализирует текст, выделяет ключевые фрагменты и предоставляет ответ на заданный вопрос.

3. **Как применяется в задаче?**  
   Задаётся вопрос, который помогает извлечь объект из текста описания. Например:
   - Вопрос: "Какой объект упоминается в задаче?"  
   Для каждого текста модель возвращает ключевой объект, связанный с целью поручения.



In [ ]:
#Инициализация модели
description_answering_pipline = pipeline(task="question-answering", model="ruselkomp/deep-pavlov-full")

### Формирование вопросов на основе кластеров

Для каждого поручения на вход подаётся его описание. В зависимости от принадлежности цели задачи к определённому кластеру, формируется соответствующий вопрос, который помогает извлечь из текста ключевой объект. Этот подход позволяет задать более точный и релевантный контекст для модели "вопрос-ответ".

#### Логика формирования вопросов

1. **Кластер "Спасти-Освободить"**  
   Если цель задачи относится к кластеру "спасти-освободить", задаётся вопрос:  
   **"Кто напал на тех, кого надо освободить или спасти?"**  
   Этот вопрос помогает выявить нападающих (например, монстров или разбойников), чтобы уточнить, от кого необходимо защитить жертв.

2. **Кластер "Вернуть-Найти"**  
   Для задач из этого кластера формируется вопрос:  
   **"Что нужно вернуть или найти?"**  
   Такой подход позволяет определить, какой именно объект требуется обнаружить (например, драгоценности, рюкзак или документы).

3. **Другие кластеры**  
   Если цель задачи не относится к первым двум кластерам (например, "убить-победить-уничтожить" или "прогнать-проучить"), задаётся вопрос:  
   **"Кто опасен и кто нападает?"**  
   Это позволяет определить угрозу или объект, от которого нужно избавиться (например, монстра, зверя или разбойников).

#### Пример работы
Каждое описание поручения анализируется индивидуально, а соответствующий вопрос помогает модели точно определить ключевой объект. Это делает процесс более гибким и универсальным, адаптированным под каждый тип задачи.


In [89]:
def get_object_description(model, description, purpose):
    if purpose == "спасти-освободить":
        question_ = "Кто напал на тех, кого надо освободить-спасти?"
    elif purpose == "вернуть-найти":
        question_ = "Что нужно вернуть-найти?"
    else: question_ = "Кто опасен и кто нападает?"

    return model(
        question=question_,
        context=description
    )["answer"]

In [90]:
description_objects = [get_object_description(
        description_answering_pipline,
        descp,
        purp
    )
    for descp, purp in df_cases[["description", "common_purpose"]].values]


После того как были получены **объекты**, необходимо их также нормализовать и лемматизировать:

In [91]:
df_cases["description_object"] = description_objects
df_cases["description_object"] = df_cases["description_object"].apply(text_preprocessor.normalize_and_lemmatize)

### Структурирование поручений на основе целей и объектов

В результате анализа удалось выделить основные кластеры поручений, сгруппированные по их целям и объектам. Этот подход позволил систематизировать задачи и придать данным чёткую структуру:

1. **Кластеры целей:**  
   Задачи разделены на группы по их целям, такие как "убить-победить-уничтожить", "спасти-освободить", "вернуть-найти", "прогнать-проучить".

2. **Определение объектов:**  
   Для каждой цели извлечены ключевые объекты, такие как "монстр", "рюкзак", "дракон". Это помогло уточнить контекст поручений и их специфику.

#### Обработка выбросов

В процессе анализа был выявлен выброс: описание "звери нападают на человека". Этот случай не соответствует стандартным кластерам и был обработан вручную для корректного включения в данные.

#### Автоматизация нормализации

После извлечения объектов выполнена нормализация данных для унификации текста. Этот шаг обеспечил единообразие описаний и упростил дальнейшую обработку.



In [94]:
df_cases[["common_purpose"	,"description_object"]].value_counts()

common_purpose             description_object        
спасти-освободить          монстр                        67
убить-победить-уничтожить  дракон                        60
вернуть-найти              драгоценность                 56
убить-победить-уничтожить  монстр                        54
                           огромный каменный чудовище    53
вернуть-найти              рюкзак                        52
                           сумка документ                48
прогнать-проучить          разбойник                     39
убить-победить-уничтожить  зверь                         29
прогнать-проучить          зверь                         23
убить-победить-уничтожить  разбойник                     19
Name: count, dtype: int64

Замена выброса:

In [95]:
df_cases["description_object"] = df_cases["description_object"].replace("звери нападать человек", "зверь")

### Анализ ценообразования

На основе предоставленных данных проведён анализ ценообразования для различных комбинаций кластеров целей и объектов. Основное внимание уделено тому, как средние значения (mean) и медианы (median) варьируются в зависимости от целей и объектов.

---

#### Варьирование средних значений

- **Максимальная средняя стоимость** наблюдается для задач:
  - Убить монстра: **18,259**.
  - Прогнать разбойников: **18,065**.
- **Минимальная средняя стоимость**:
  - Убить разбойников: **14,394**.

Разница между самой дорогой и самой дешёвой средней стоимостью составляет около **3,865**, что указывает на умеренную вариацию.

#### Варьирование медиан

- Медианы имеют меньшее разброс:
  - Максимальная медиана: убить монстра (18,500).
  - Минимальная медиана: убить разбойников (13,000).
- Разница между максимальной и минимальной медианой составляет **5,500**, что указывает на сильное влияние объекта на восприятие ценности задачи.

---

### Обнаруженные зависимости

1. **Цели "убить-победить-уничтожить"**
   - Стоимость задач зависит от объекта:
     - Убийство разбойников стоит существенно меньше (mean: 14,394), чем убийство монстров (mean: 18,259).
     - Убийство дракона и зверя имеют схожую стоимость (около 16,400).

2. **Цели "вернуть-найти"**
   - Ценности задач на поиск вещей зависят от объекта:
     - Драгоценности имеют максимальную среднюю стоимость (18,040).
     - Рюкзак и сумка с документами оцениваются дешевле, но их стоимость близка (mean около 15,800–17,400).

3. **Цели "прогнать-проучить"**
   - Разбойники имеют более высокую стоимость (mean: 18,065) по сравнению со зверями (mean: 16,022).

4. **Цели "спасти-освободить"**
   - Монстры оцениваются относительно высоко (mean: 16,782), что сравнимо со стоимостью убийства более опасных объектов.

---

### Выводы

- **Сильная зависимость от объекта:** Стоимость задачи варьируется значительно в зависимости от её цели и объекта. Например, убийство монстра стоит значительно дороже, чем убийство разбойников, а возврат драгоценностей ценится выше, чем возврат рюкзаков.
- **Медианы стабилизируют распределение:** Несмотря на различия в средних значениях, медианы указывают на меньшее варьирование внутри групп задач.
- **Цели "убить" и "прогнать" наиболее подвержены изменению стоимости в зависимости от сложности объекта.**

Общая структура ценообразования относительно стабильна, однако объекты оказывают значительное влияние на итоговую оценку задачи.

### Зависимость от заказчика

Поскольку медианы не сильно разбросаны, можно сделать вывод, что влияние заказчика на цену задачи незначительно. Цены преимущественно зависят от цели и объекта задачи, а не от того, кто сделал заказ.

In [96]:
cluster_stats = df_cases[df_cases["Выполнено"]=="да"].groupby(["common_purpose", "description_object"])['Сумма вознаграждения'].agg(['mean', 'median']).reset_index()
cluster_stats

,common_purpose,description_object,mean,median
0,вернуть-найти,драгоценность,18040.000000,15750.0
1,вернуть-найти,рюкзак,17400.000000,16500.0
2,вернуть-найти,сумка документ,15885.416667,15750.0
3,прогнать-проучить,зверь,16022.727273,16250.0
4,прогнать-проучить,разбойник,18065.789474,20750.0
5,спасти-освободить,монстр,16782.258065,15500.0
6,убить-победить-уничтожить,дракон,16635.593220,16500.0
7,убить-победить-уничтожить,зверь,16428.571429,17500.0
8,убить-победить-уничтожить,монстр,18259.259259,18500.0
9,убить-победить-уничтожить,огромный каменный чудовище,16186.274510,15500.0


## 1.2 Дневники + оценки потребителей

In [97]:
df_diaries = pd.read_csv("../data/diaries.csv", delimiter=";", encoding='utf-8-sig', decimal=",")

In [98]:
df_diaries.head(4)

,Номер поручения,Герой,Запись в дневнике,Затрачено часов,Роль
0,11000,Мартин,разжечь костёр,1.0,рейнджер
1,11000,Мартин,выследить цель,6.0,следопыт
2,11001,Альфред,разжечь костёр,1.0,рейнджер
3,11001,Альфред,залечить раны,18.0,лекарь




#### Структурированность и ответственность героев

На основании анализа дневниковых записей можно сделать вывод, что герои подходят к документированию своих задач крайне ответственно. Все записи выполняются без использования синонимов, в строго унифицированной форме. Например, действие "разжечь костёр" фиксируется одинаково, независимо от героя или контекста. Это позволяет легко интерпретировать и классифицировать их задачи.



#### Оценка задач по ролям героев

Для анализа задач было использовано агрегирование по героям, их ролям и записям в дневнике. Этот подход позволил оценить, какие действия могут выполнять представители каждой роли. 

#### Выводы

1. **Унифицированные записи**  
   Все действия фиксируются героически четко и без вариаций в формулировках. Это упрощает анализ их работы и позволяет исключить двусмысленности.

2. **Распределение задач по ролям**  
   Каждая роль выполняет специфические задачи:
   - **Рейнджеры, Лучники, Боевые маги, Мечники:** Выполняют задания, связанные с поддержанием выживания, такие как "разжечь костёр".
   - **Следопыты:** Занимаются поисковыми задачами, такими как "выследить цель", "найти пропажу", "отыскать заказ".
   - **Лекари:** Сосредотачиваются на помощи команде, например, "залечить раны".

3. **Агрегация данных**  
   Сгруппированные данные позволяют понять вклад каждого героя и его роли в выполнении поручений, а также оценить затраты времени на разные действия.




In [99]:
df_roles_tasks = df_diaries.groupby(["Герой", "Роль"])["Запись в дневнике"].apply(set).reset_index()
df_roles_tasks["Запись в дневнике"] = df_roles_tasks["Запись в дневнике"].map(sorted).map(tuple)

df_roles_skills = df_roles_tasks.groupby('Роль')['Запись в дневнике'].unique().reset_index()
df_roles_skills["Уникальное число способностей"] = df_roles_skills["Запись в дневнике"].apply(len)

In [100]:
df_roles_skills["Запись в дневнике"] = df_roles_skills["Запись в дневнике"].map(lambda x: list(x[0]))

In [101]:
df_roles_skills

,Роль,Запись в дневнике,Уникальное число способностей
0,боевой маг,[разжечь костёр],1
1,лекарь,[залечить раны],1
2,лучник,[разжечь костёр],1
3,мечник,[разжечь костёр],1
4,рейнджер,[разжечь костёр],1
5,следопыт,"[выследить цель, найти пропажу, отыскать заказ...",1


Рассмотрим оценки героев

In [102]:
df_marks = pd.read_csv("../data/marks.csv", delimiter=";", encoding='utf-8-sig', decimal=",")


In [103]:
df_marks.head()

,Номер поручения,Герой,Оценка за качество,Оценка по срокам,Оценка за вежливость
0,11000,Мартин,4,3,4
1,11001,Альфред,5,5,4
2,11002,Мартин,5,4,4
3,11003,Бендер,2,4,3
4,11004,Юлия,4,4,5


### Анализ оценок героев

При анализе средних оценок героев без учёта их ролей и кластеров можно заметить, что значения для большинства показателей находятся в узком диапазоне. Это приводит к минимальным различиям между героями, что затрудняет выделение их уникальных характеристик.

---

#### Наблюдения

1. **Близость средних оценок**  
   Все оценки за качество, сроки и вежливость находятся в диапазоне **3.67–4.00**, что свидетельствует о схожем уровне выполнения задач среди героев. Например:
   - **Оценка за качество** варьируется от **3.78** (Бенедикт) до **4.00** (Пастушок).
   - **Оценка по срокам** колеблется от **3.67** (Юлия) до **4.00** (Фредерик).
   - **Оценка за вежливость** имеет небольшой разброс от **3.83** (Соня) до **4.00** (Глюкоза, Пастушок).

2. **Отсутствие значимой дифференциации**  
   Из-за близости значений средних оценок становится сложно определить сильные и слабые стороны каждого героя. Это снижает ценность анализа, если не учитывать дополнительные факторы.

---

#### Учёт ролей и кластеров

Чтобы сделать анализ более значимым, необходимо учитывать:
- **Роли героев:** Разные роли предполагают выполнение различных типов задач, что может влиять на их оценки. Например, от рейнджеров могут ожидать более высоких оценок по срокам, а от лекарей — по вежливости.
- **Кластеры задач:** Учитывая специфику поручений, можно оценивать героев не только в общем, но и по средним показателям в их ключевых задачах.

---

#### Преимущества кластеризации

- **Повышение точности анализа:** Учитывая роли и кластеры, можно выявить сильные стороны героев в их специализированных задачах.
- **Выявление слабых мест:** Анализ по ролям помогает понять, в каких аспектах конкретные герои нуждаются в улучшении.
- **Целевая обратная связь:** Средние оценки по ролям дают более точную информацию для развития каждого героя.

Таким образом, без учёта ролей и кластеров средние оценки теряют аналитическую ценность, что делает кластеризацию необходимым этапом анализа.


In [104]:
df_mean_scores_heroes = df_marks.groupby(["Герой"]).agg(
    {
        "Оценка за качество": "mean",
        "Оценка по срокам": "mean",
        "Оценка за вежливость": "mean"
    }
).reset_index()

In [105]:
df_mean_scores_heroes

,Герой,Оценка за качество,Оценка по срокам,Оценка за вежливость
0,Агата,3.939394,3.727273,4.000000
1,Альфред,3.871795,3.692308,3.897436
2,Бендер,3.928571,3.928571,3.928571
3,Бенедикт,3.785714,3.880952,3.928571
4,Глюкоза,3.863636,3.931818,4.000000
5,Леопольд,3.893617,3.808511,3.872340
6,Мартин,3.967742,3.935484,3.935484
7,Пастушок,4.000000,3.836735,4.000000
8,Синеглазый,3.878049,3.756098,3.878049
9,Соня,3.888889,3.972222,3.833333


### Агрегация данных по ролям

Для более глубокого анализа был проведён процесс агрегации данных, позволяющий определить, какие роли выполнял каждый герой в различных поручениях. Этот подход помогает связать действия героев с их ролями и оценить их вклад в выполнение задач.

---

#### Задачи агрегации

1. **Выявление ролей героев**  
   Сгруппированы данные, чтобы увидеть, как герои распределяли свои усилия между разными ролями. Например:
   - Один герой мог выступать как рейнджер в одном поручении и как лекарь в другом.
   - Выявлены специфические задачи, наиболее часто выполняемые каждой ролью.

2. **Связь ролей с поручениями**  
   Для каждого поручения определено, какие роли были задействованы. Это позволяет понять, какие именно действия выполнялись в зависимости от задачи.

#### Выводы

1. **Разнообразие ролей**  
   Герои часто выполняют несколько ролей в рамках одного или разных поручений.

2. **Оптимизация ресурсов**  
   Агрегация помогает выявить, какие роли требуются для выполнения различных задач, что способствует более эффективному распределению усилий.

3. **База для дальнейшего анализа**  
   Эта структура данных позволяет изучить:
   - Средние показатели героев в каждой роли.
   - Зависимость успеха поручения от распределения ролей.

Агрегация данных по ролям оказалась полезным инструментом для понимания вклада героев в выполнение поручений и дальнейшей оптимизации их работы.


In [106]:
df_diaries_heroes_agg = df_diaries.groupby(['Номер поручения', 'Герой']).agg({'Роль': lambda x: list(x)}).reset_index()
df_diaries_heroes_agg["solo"] = df_diaries_heroes_agg["Роль"].apply(lambda x: 1 if len(x) == 1 else 0)

In [107]:
df_diaries_heroes_agg

,Номер поручения,Герой,Роль,solo
0,11000,Мартин,"[рейнджер, следопыт, лучник]",0
1,11001,Альфред,"[рейнджер, лекарь, мечник]",0
2,11002,Мартин,[следопыт],1
3,11003,Бендер,[мечник],1
4,11004,Юлия,"[следопыт, следопыт]",0
...,...,...,...,...
476,11495,Синеглазый,"[рейнджер, лекарь, мечник]",0
477,11496,Юлия,"[рейнджер, лекарь, мечник]",0
478,11497,Пастушок,"[рейнджер, мечник]",0
479,11498,Мартин,"[рейнджер, лучник]",0


### Обогащение данных для дальнейшего анализа

После агрегации данных по ролям к полученной таблице были добавлены дополнительные данные, включая оценки и кластеры (цель и объект), к которым относится каждое поручение. Это позволило значительно расширить возможности анализа и выявить новые закономерности.

---

#### Было добавлено:

1. **Оценки выполнения задач:**  
   Для каждой записи добавлены средние оценки героев:
   - **Оценка за качество.**
   - **Оценка по срокам.**
   - **Оценка за вежливость.**  
   Эти данные позволяют оценить эффективность выполнения поручений в разрезе ролей и кластеров.

2. **Кластеризация поручений:**  
   Каждое поручение было отнесено к одному из кластеров, определённых ранее:
   - Цель задачи (например, "убить-победить-уничтожить", "вернуть-найти").
   - Объект задачи (например, "монстр", "рюкзак", "дракон").  
   Это добавление позволяет более точно анализировать, как различные роли справляются с разными типами задач.


#### Преимущества добавления данных

1. **Анализ производительности по ролям и задачам:**  
   Теперь можно определить, как конкретные роли справляются с определёнными целями и объектами задач. Например, рейнджеры могут иметь более высокие оценки за выполнение задач, связанных с разведкой, чем с боевыми действиями.

2. **Выявление сильных и слабых сторон:**  
   Дополнительные данные помогают анализировать, какие роли и герои лучше всего подходят для выполнения конкретных типов поручений.

3. **Обоснованное распределение ролей:**  
   С учётом оценок и кластеров становится возможным предсказать, какие роли и герои будут наиболее эффективны в будущих поручениях.



Добавление оценок и кластеров сделало таблицу более информативной и позволило подготовить данные для дальнейшего анализа и оптимизации распределения задач между героями.


In [108]:
df_heroes_w_marks = pd.merge(
    pd.merge(df_diaries_heroes_agg, 
         df_cases[df_cases["Выполнено"]=="да"][["Номер поручения", "common_purpose", "description_object"]],
         on="Номер поручения",
         how = "outer"),
    df_marks,
    on="Номер поручения",
    how="outer"
)

In [109]:
df_heroes_w_marks.head(5)

,Номер поручения,Герой_x,Роль,solo,common_purpose,description_object,Герой_y,Оценка за качество,Оценка по срокам,Оценка за вежливость
0,11000,Мартин,"[рейнджер, следопыт, лучник]",0.0,убить-победить-уничтожить,зверь,Мартин,4,3,4
1,11001,Альфред,"[рейнджер, лекарь, мечник]",0.0,убить-победить-уничтожить,разбойник,Альфред,5,5,4
2,11002,Мартин,[следопыт],1.0,спасти-освободить,монстр,Мартин,5,4,4
3,11003,Бендер,[мечник],1.0,убить-победить-уничтожить,монстр,Бендер,2,4,3
4,11004,Юлия,"[следопыт, следопыт]",0.0,вернуть-найти,сумка документ,Юлия,4,4,5


### Учет сложности оценки ролей при многозадачности

Анализ показал, что **3/4 поручений (316 из 434)** выполнялись героями, которые одновременно занимали несколько ролей. Это создает трудности при точной оценке каждой роли отдельно, так как оценки за поручение распределяются на несколько ролей.  

---

### Агрегация данных по ролям, кластерам и оценкам

Чтобы корректно оценить вклад героев в выполнение задач, были выполнены следующие шаги:
1. **Агрегация данных**  
   Для каждого героя были рассчитаны средние оценки по:
   - Его ролям.
   - Кластерам (цель-объект).
   - Поручениям.

2. **Обработка поручений с несколькими ролями**  
   Если герой выполнял поручение в нескольких ролях, то для каждой роли отдельно учитывались оценки из этих поручений.

3. **Результат: таблица `df_roles`**  
   Итогом стала таблица, которая содержит информацию о каждом герое, его ролях, связанных кластерах, и средних оценках. 

---

### Отбор лучших героев

Для дальнейшего анализа из таблицы `df_roles` были отобраны только лучшие герои, у которых одновременно:
- **Средние оценки по срокам больше 2.**
- **Средние оценки по качеству больше 3.**

Этот фильтр гарантирует, что выбранные герои способны эффективно выполнять задачи.

---

### Создание профилей героев

На основе данных из `df_roles` была создана таблица **`hero_profiles`**, которая содержит следующую информацию:
- **Герой:** Имя героя.
- **Роль:** Роль героя в поручении.
- **Кластер:** К какому кластеру (цель-объект) относятся выполненные задачи.
- **Средние оценки:** Средние значения по качеству, срокам и вежливости.
- **Действия из дневника:** Список задач, выполненных героем в данной роли.
- **Общее время:** Сумма затраченных часов на выполнение задач.


---

### Итог

Сформированные данные позволяют:
- Точно оценить эффективность героев в их ролях.
- Провести детализированный анализ задач, связанных с конкретными кластерами.
- Упростить планирование задач и распределение ролей в будущем.


In [ ]:
#1 - если герой выступал в качестве одной роли
df_heroes_w_marks["solo"].value_counts()

solo
0.0    347
1.0    134
Name: count, dtype: int64

In [111]:
df_heroes_w_marks['Роль'] = df_heroes_w_marks['Роль'].apply(lambda x: x if isinstance(x, list) else [x])
df_single_role = df_heroes_w_marks[df_heroes_w_marks['Роль'].apply(len) == 1].explode('Роль', ignore_index=True)
df_multi_role = df_heroes_w_marks[df_heroes_w_marks["Роль"].isin(df_single_role["Роль"]) == False].explode('Роль', ignore_index=True)
df_expanded = pd.concat([df_single_role, df_multi_role])



df_roles = df_expanded.groupby(["Герой_x", "Роль", "common_purpose",	"description_object"], as_index=False).agg(
    {
        "Оценка за качество": "mean",
        "Оценка по срокам": "mean",
        "Оценка за вежливость": "mean"
    }
).reset_index()

In [112]:
df_roles = df_roles[(
    df_roles["Оценка за качество"] >3) & (df_roles["Оценка по срокам"] > 2)]
df_roles = df_roles.rename(columns={'Герой_x': 'Герой'})

In [113]:
df_roles.head(5)

,index,Герой,Роль,common_purpose,description_object,Оценка за качество,Оценка по срокам,Оценка за вежливость
0,0,Агата,лучник,прогнать-проучить,зверь,4.0,3.000000,4.000000
1,1,Агата,лучник,прогнать-проучить,разбойник,4.0,3.333333,4.666667
2,2,Агата,лучник,убить-победить-уничтожить,дракон,4.2,4.200000,4.000000
3,3,Агата,лучник,убить-победить-уничтожить,зверь,4.0,3.333333,5.000000
4,4,Агата,лучник,убить-победить-уничтожить,монстр,3.5,3.500000,4.000000


In [114]:
hero_profiles = pd.merge(
    df_roles,
    pd.merge(
        df_diaries,
        df_cases[["Номер поручения", "common_purpose", "description_object"]],
        on="Номер поручения",
        how="left"
    ).groupby(
        ["Герой", "Запись в дневнике", "Роль", "common_purpose", "description_object"]
        )["Затрачено часов"].mean().reset_index(),
    on=["Герой", "Роль", "common_purpose", "description_object"],
    how="left"        
).drop(columns="index")

In [115]:
hero_profiles.head(5)

,Герой,Роль,common_purpose,description_object,Оценка за качество,Оценка по срокам,Оценка за вежливость,Запись в дневнике,Затрачено часов
0,Агата,лучник,прогнать-проучить,зверь,4.0,3.000000,4.000000,разжечь костёр,0.666667
1,Агата,лучник,прогнать-проучить,разбойник,4.0,3.333333,4.666667,разжечь костёр,0.666667
2,Агата,лучник,убить-победить-уничтожить,дракон,4.2,4.200000,4.000000,разжечь костёр,0.666667
3,Агата,лучник,убить-победить-уничтожить,зверь,4.0,3.333333,5.000000,разжечь костёр,0.666667
4,Агата,лучник,убить-победить-уничтожить,монстр,3.5,3.500000,4.000000,разжечь костёр,0.666667


После чего для дальнешего упрощения работы колонки кластеров **цель** и **объект** были объединены в один с помощью метода ``connect_clusters``

In [116]:
def connect_clusters(row):
    return " ".join(row[["common_purpose", "description_object"]])

In [117]:
hero_profiles["cluster"] = hero_profiles.apply(connect_clusters, axis=1, result_type="expand")

In [118]:
hero_profiles.head(5)

,Герой,Роль,common_purpose,description_object,Оценка за качество,Оценка по срокам,Оценка за вежливость,Запись в дневнике,Затрачено часов,cluster
0,Агата,лучник,прогнать-проучить,зверь,4.0,3.000000,4.000000,разжечь костёр,0.666667,прогнать-проучить зверь
1,Агата,лучник,прогнать-проучить,разбойник,4.0,3.333333,4.666667,разжечь костёр,0.666667,прогнать-проучить разбойник
2,Агата,лучник,убить-победить-уничтожить,дракон,4.2,4.200000,4.000000,разжечь костёр,0.666667,убить-победить-уничтожить дракон
3,Агата,лучник,убить-победить-уничтожить,зверь,4.0,3.333333,5.000000,разжечь костёр,0.666667,убить-победить-уничтожить зверь
4,Агата,лучник,убить-победить-уничтожить,монстр,3.5,3.500000,4.000000,разжечь костёр,0.666667,убить-победить-уничтожить монстр


In [336]:
#Профили героев
hero_profiles.to_csv("result_heroes_profiles_clusters.csv")

### Полученные в результате анализа таблицы позволяют перейти к следующему этапу

# 2. Подготовка к модели

## 2.1 Таблица поручений

Объединение колонок кластеров **цель** и **объект**  в один с помощью метода ``connect_clusters``

In [120]:
df_cases["cluster"] = df_cases.apply(connect_clusters, axis=1, result_type="expand")

### Учет временного фактора для рекомендательной системы

Для построения качественной рекомендательной системы важно учитывать **временной фактор** выполнения поручений, чтобы анализировать доступность героев в разные промежутки времени. Это позволяет оптимально распределять задачи и предсказывать участие героев в будущем.

---

#### Преобразование дат для анализа

1. **Метод `convert_date`**  
   Для работы с временными данными был разработан метод `convert_date`, который преобразует даты в числовой формат, удобный для дальнейшего анализа.

2. **Определение диапазона работы гильдии**  
   На основе списка поручений были вычислены:
   - **Минимальная дата (`min_date`)**: начало работы гильдии.
   - **Максимальная дата (`max_date`)**: крайняя дата выполнения или появления поручений.

3. **Замена дат на числовые значения**  
   Каждая дата в таблице была заменена на соответствующее число, где:
   - **Дата поручения_t**: дни с начала горизонта планирования (от `1` до `n`).
   - **Дата выполнения_t**: дни с начала горизонта планирования (от `1` до `n`).

#### Горизонт планирования

Полученный список дней от 1 до `n` называется **горизонтом планирования**, где:
- `n = max_date - min_date` (в днях).
- Это последовательность, используемая для анализа временной доступности героев.



#### Зачем это нужно?

1. **Анализ доступности героев**  
   Числовой формат позволяет легко определить, кто из героев был занят в определённый промежуток времени.

2. **Горизонт планирования**  
   Универсальная шкала времени упрощает прогнозирование и планирование задач в будущем.

3. **Интеграция в рекомендательную систему**  
   Использование временного фактора в сочетании с ролями, кластерами и оценками позволяет повысить точность рекомендаций.

---

### Итог

Преобразование данных в числовой формат с использованием метода `convert_date` и формирование горизонта планирования создали основу для учета временного фактора в рекомендательной системе. Это улучшает качество анализа и планирования задач в рамках работы гильдии.


In [ ]:
def convert_date(date):
    if pd.notna(date):
        try:
            return datetime.strptime(date, "%Y-%m-%d")
        except ValueError:
            return date
    return date

In [ ]:
#Преобразование строковых дат в объект datetime
df_cases["time_in"] = df_cases["Дата поручения"].apply(convert_date)
df_cases["time_finish"] = df_cases["Дата выполнения"].apply(convert_date)

In [ ]:
#Поиск минимальной и максимальной даты среди дат поступления поручений и их конца выполнения
min_date = min(df_cases["time_in"].min(),df_cases["time_finish"].dropna().min())
max_date = max(df_cases["time_in"].max(),df_cases["time_finish"].dropna().max())

In [ ]:
#Создание словаря замены даты в кванты времени и наоборот
t_date_list = []
date_t_list = []
current_date = min_date
i = 0
while current_date <= max_date:
    t_date_list.append([i:= i +1 ,current_date])
    date_t_list.append([current_date, i])
    current_date += timedelta(days=1)

t_date_dict = dict(t_date_list)
date_t_dict = dict(date_t_list)

In [ ]:
#Перевод дат в кванты времени (у невыполненных поручений Nan заменен на -1, чтобы код отработал)
df_cases["Дата поручения_t"] = df_cases["time_in"].map(date_t_dict)
df_cases["Дата выполнения_t"] = df_cases["time_finish"].map(date_t_dict).fillna(-1).astype(int)

In [ ]:
#Горизонт планирования
t = np.array(range(1, date_t_list[-1][-1]))

### Создание обучающей выборки на основе исторических данных

Для разработки качественной модели рекомендательной системы была создана обучающая выборка, основанная на исторических данных о выполненных поручениях.

---

#### Источник данных

1. **Таблица выполненных поручений**  
   В выборку были включены только те поручения, которые помечены как успешно выполненные.

2. **Объединение данных**  
   Выполненные поручения были объединены с:
   - **Записями из дневника:** Описания задач, которые герой выполнял для выполнения поручения.
   - **Оценками по срокам:** Показатель, отражающий удовлетворённость заказчика скоростью выполнения задачи.

---

#### Подход к оценке времени выполнения

Для анализа временного фактора было сделано допущение:  
**Оценка по срокам пропорциональна количеству времени, затраченного героем на выполнение задачи.**

- **Интерпретация оценки:**  
  Например, если оценка по срокам равна `3`, это указывает на то, что заказчик ожидал более быстрого выполнения задачи.

- **Формула подсчета идеального времени выполнения:**  
  На основе этой идеи была выведена формула:  
  
  * **Идеальное время выполнения = ceil(Фактическое время выполнения * Оценка по срокам / 5)**
   
  Где:
  - **Фактическое время выполнения** — количество дней, которое герой реально затратил.
  - **Оценка по срокам** — оценка заказчика по шкале от 1 до 5.
  - **Округление вверх** ``ceil`` связано с тем, что задачи оцениваются в целых днях.

Оценка идеального времени для задачи реализована с помощью метода ``ideal_time``

---

#### Зачем это нужно?

1. **Формирование эталона времени выполнения**  
   Идеальное время выполнения позволяет установить ориентир для оценки, за сколько фактических дней команда выполняет те или иные задачи.

2. **Анализ производительности**  
   Сравнение фактического времени с идеальным помогает выявить слабые места и улучшить планирование.

3. **Улучшение рекомендательной системы**  
   Использование идеального времени в качестве целевой метрики делает модель более адаптированной к ожиданиям заказчиков.

---

### Итог

На основе исторических данных о выполненных задачах и оценок по срокам была создана обучающая выборка. Формула подсчета идеального времени выполнения позволяет использовать временной фактор для оптимизации выполнения задач и повышения удовлетворённости заказчиков.



In [127]:
def ideal_time(row):
    return int(np.ceil(row["Оценка по срокам"] * row["Затрачено дней"] / 5))

In [128]:
df_train_cases = pd.merge(
    df_cases[df_cases["Выполнено"]=="да"],
    pd.merge(
        df_diaries.groupby(["Номер поручения", "Герой"])["Запись в дневнике"].apply(list).reset_index(),
        df_marks[["Номер поручения", "Оценка по срокам"]],
        on=["Номер поручения"]
    ),
    on="Номер поручения"
)

In [129]:
df_train_cases.head(5)

,Номер поручения,Заказчик,Дата поручения,Выполнено,Дата выполнения,Затрачено дней,Сумма вознаграждения,Описание,description,advice,...,common_purpose,description_object,cluster,time_in,time_finish,Дата поручения_t,Дата выполнения_t,Герой,Запись в дневнике,Оценка по срокам
0,11000,Анна,1051-08-11,да,1051-08-21,4.0,6000,В лесу по дороге от пещеры звери нападают на л...,В лесу по дороге от пещеры звери нападают на л...,"Осмотрите все возможные укрытия, чтобы найти з...",...,убить-победить-уничтожить,зверь,убить-победить-уничтожить зверь,1051-08-11 00:00:00,1051-08-21 00:00:00,221,231,Мартин,"[разжечь костёр, выследить цель, разжечь костёр]",3
1,11001,Мария,1051-07-09,да,1051-09-02,2.0,20000,В лесу по дороге от пещеры были замечены разбо...,В лесу по дороге от пещеры были замечены разбо...,"Будьте осторожны, так как разбойники могут быт...",...,убить-победить-уничтожить,разбойник,убить-победить-уничтожить разбойник,1051-07-09 00:00:00,1051-09-02 00:00:00,188,243,Альфред,"[разжечь костёр, залечить раны, разжечь костёр]",5
2,11002,Эмилио,1053-10-05,да,1053-10-20,7.0,22500,По дороге из деревни монстры похитили путников...,По дороге из деревни монстры похитили путников,"Осмотрите все возможные укрытия, чтобы найти п...",...,спасти-освободить,монстр,спасти-освободить монстр,1053-10-05 00:00:00,1053-10-20 00:00:00,1007,1022,Мартин,[выследить цель],4
3,11003,Бьянка,1052-11-24,да,1052-12-02,5.0,5500,Недалеко от города видели монстров. Нужно побе...,Недалеко от города видели монстров,"Будьте осторожны, так как монстры могут быть о...",...,убить-победить-уничтожить,монстр,убить-победить-уничтожить монстр,1052-11-24 00:00:00,1052-12-02 00:00:00,692,700,Бендер,[разжечь костёр],4
4,11004,Бьянка,1052-02-23,да,1052-03-30,8.0,10500,В деревне у меня пропала сумка с документами. ...,В деревне у меня пропала сумка с документами,,...,вернуть-найти,сумка документ,вернуть-найти сумка документ,1052-02-23 00:00:00,1052-03-30 00:00:00,417,453,Юлия,"[найти пропажу, отыскать заказчика]",4


In [130]:
df_train_cases["Идеальное время"] = df_train_cases.apply(ideal_time, axis=1, result_type="expand")
df_train_cases["Затрачено дней"] = df_train_cases["Затрачено дней"].astype(int)

Удаление лишних неинформативных далее колонок

In [131]:
df_train_cases = df_train_cases[["Номер поручения", "Заказчик", "Дата поручения", 
                "Дата выполнения", "Затрачено дней", "Сумма вознаграждения",
                "description", "cluster", "time_in", "time_finish", "Дата поручения_t",
                "Дата выполнения_t", "Герой", "Запись в дневнике", "Идеальное время"
                ]]

### Расчет квантов времени для задач и героев

Для анализа временной доступности задач и героев был разработан подход, основанный на расчёте **квантов времени**, которые определяют:

1. **Доступность задачи для выполнения**  
   Каждый кванты времени для задачи определяется как промежуток между:
   - **Датой появления поручения в гильдии** (дата начала).
   - **Максимальной доступной датой** (граничный кванты времени для планирования).

2. **Доступность героев**  
   Для каждого героя учитываются его занятость и пересечения между:
   - **Выполненными поручениями**.
   - **Поручениями, которые находятся в процессе выполнения.**

Эти данные позволяют учитывать, что некоторые задачи могут накладываться на невыполненные или параллельно выполняемые поручения.

---

#### Построение квантов времени

Для построения квантов времени была использована функция **`create_date_range`**, которая создаёт последовательность дат, представляющих доступность:

1. **Для задач:**  
   Кванты времени создаются между датой появления задачи в гильдии и максимальной доступной датой (горизонт планирования).

2. **Для героев:**  
   Кванты времени формируются на основе:
   - Даты завершения последнего поручения героя.
   - Возможных пересечений с другими задачами.




In [132]:
def create_date_range(row):
    return [row["Дата выполнения_t"] - i for i in range(0, row["Дата выполнения_t"] - row["Идеальное время"])]

In [133]:
df_train_cases['диапазон дат'] = df_train_cases.apply(create_date_range, axis=1)

In [134]:
grouped_df = df_train_cases[["Герой", "диапазон дат"]].groupby('Герой')['диапазон дат'].sum().reset_index()
grouped_df['диапазон дат'] = grouped_df['диапазон дат'].apply(lambda x: sorted(set(x)))
grouped_df["feasible_date"] = grouped_df["диапазон дат"].apply(lambda x: list(np.setdiff1d(t, np.array(x))))

heroes_availibility = dict(grouped_df[["Герой", "feasible_date"]].values)

In [135]:
heroes_availibility

{'Агата': [1,
  1055,
  1056,
  1057,
  1058,
  1059,
  1060,
  1061,
  1062,
  1063,
  1064,
  1065,
  1066,
  1067,
  1068,
  1069,
  1070,
  1071,
  1072,
  1073,
  1074,
  1075,
  1076,
  1077,
  1078,
  1079,
  1080,
  1081,
  1082,
  1083,
  1084,
  1085,
  1086,
  1087,
  1088,
  1089,
  1090,
  1091,
  1092,
  1093,
  1094,
  1095,
  1096,
  1097,
  1098,
  1099,
  1100,
  1101,
  1102,
  1103,
  1104,
  1105,
  1106],
 'Альфред': [1,
  1044,
  1045,
  1046,
  1047,
  1048,
  1049,
  1050,
  1051,
  1052,
  1053,
  1054,
  1055,
  1056,
  1057,
  1058,
  1059,
  1060,
  1061,
  1062,
  1063,
  1064,
  1065,
  1066,
  1067,
  1068,
  1069,
  1070,
  1071,
  1072,
  1073,
  1074,
  1075,
  1076,
  1077,
  1078,
  1079,
  1080,
  1081,
  1082,
  1083,
  1084,
  1085,
  1086,
  1087,
  1088,
  1089,
  1090,
  1091,
  1092,
  1093,
  1094,
  1095,
  1096,
  1097,
  1098,
  1099,
  1100,
  1101,
  1102,
  1103,
  1104,
  1105,
  1106],
 'Бендер': [1,
  1036,
  1037,
  1038,
  1039,
 

In [136]:
df_train_cases.head()

,Номер поручения,Заказчик,Дата поручения,Дата выполнения,Затрачено дней,Сумма вознаграждения,description,cluster,time_in,time_finish,Дата поручения_t,Дата выполнения_t,Герой,Запись в дневнике,Идеальное время,диапазон дат
0,11000,Анна,1051-08-11,1051-08-21,4,6000,В лесу по дороге от пещеры звери нападают на л...,убить-победить-уничтожить зверь,1051-08-11 00:00:00,1051-08-21 00:00:00,221,231,Мартин,"[разжечь костёр, выследить цель, разжечь костёр]",3,"[231, 230, 229, 228, 227, 226, 225, 224, 223, ..."
1,11001,Мария,1051-07-09,1051-09-02,2,20000,В лесу по дороге от пещеры были замечены разбо...,убить-победить-уничтожить разбойник,1051-07-09 00:00:00,1051-09-02 00:00:00,188,243,Альфред,"[разжечь костёр, залечить раны, разжечь костёр]",2,"[243, 242, 241, 240, 239, 238, 237, 236, 235, ..."
2,11002,Эмилио,1053-10-05,1053-10-20,7,22500,По дороге из деревни монстры похитили путников,спасти-освободить монстр,1053-10-05 00:00:00,1053-10-20 00:00:00,1007,1022,Мартин,[выследить цель],6,"[1022, 1021, 1020, 1019, 1018, 1017, 1016, 101..."
3,11003,Бьянка,1052-11-24,1052-12-02,5,5500,Недалеко от города видели монстров,убить-победить-уничтожить монстр,1052-11-24 00:00:00,1052-12-02 00:00:00,692,700,Бендер,[разжечь костёр],4,"[700, 699, 698, 697, 696, 695, 694, 693, 692, ..."
4,11004,Бьянка,1052-02-23,1052-03-30,8,10500,В деревне у меня пропала сумка с документами,вернуть-найти сумка документ,1052-02-23 00:00:00,1052-03-30 00:00:00,417,453,Юлия,"[найти пропажу, отыскать заказчика]",7,"[453, 452, 451, 450, 449, 448, 447, 446, 445, ..."


## 2.2 Поиск ближайших поручений

### Генерация допустимых команд для выполнения задач

Для эффективного распределения задач между героями был создан алгоритм представленный в виде класса ``TaskManager``, который формирует **допустимые команды** на основе доступности героев, их оценок и требований к задачам.

---

#### Основные критерии формирования команд

1. **Доступность команды в квантах времени**  
   Для каждой задачи алгоритм проверяет доступность героев в интервале времени, начиная с момента поступления задачи (start_quant). Например:
   - Для каждой сгенерированной команды оценивается её способность выполнить задачу в ожидаемое время **time_task**, основанное на средней оценке по срокам героев.
   - Если задача поступила в `start_quant = 4` и её **task_time = 5**, то герои в команде должны быть доступны в интервале от `t` до `t + task_time` (включительно), для какого-либо `t` >= `start_quant`

2. **Список задач внутри задачи**  
   Для каждой задачи формируются команды, которые могут выполнить весь набор подзадач, входящих в поручение. Один герой может выполнять несколько задач внутри одного поручения, при этом в команде он будет учитываться несколько раз.

3. **Фиксированная длина команды**  
   Все команды имеют равное количество слотов, но один герой может занимать несколько слотов, если он выполняет несколько задач. Например:
   - Если герой выполняет две задачи, он считается дважды в составе команды.
   Фактически установлено ограничение на максимально кол-во уникального героя команды = 1. Но поскольку может быть один и тот же герой в разных ролях и действиях, то команды будут размером также от 1 до 4
---

#### Оценка производительности команды

После формирования команды рассчитываются её характеристики:

1. **Средние оценки команды:**
   - Средняя оценка по качеству.
   - Средняя оценка по срокам.
   - Максимальная оценка по вежливости среди членов команды, так как будем считать, что вежливость команды может представлять один герой.

2. **Средневзвешенная оценка команды**  
   Для определения общей эффективности команды рассчитывается **средневзвешенная оценка**:
   
   * **Оценка команды = 0.5 * Качество + 0.4 * Сроки + 0.1 * Вежливость **
   

---

#### Зачем это нужно?

1. **Учёт доступности героев**  
   Команды формируются с учётом времени, когда герои свободны для выполнения задачи, что минимизирует пересечения и простаивание.

2. **Оптимизация команд**  
   Генерация фиксированных команд позволяет оценить их эффективность на основе оценок и временных факторов.

3. **Повышение точности планирования**  
   Использование средневзвешенной оценки позволяет выбирать команды, которые обеспечат наилучшее выполнение задачи.

---

### Итог

Алгоритм генерации допустимых команд обеспечивает эффективное распределение задач, учитывая доступность героев, их оценки и временные ограничения. Это создаёт основу для построения более точной и качественной рекомендательной системы.


In [ ]:
class TaskManager:
    def __init__(self, heroes_table, hero_availability):
        self.heroes_table = heroes_table
        self.hero_availability = hero_availability

    def find_heroes_for_task(self, tasks):
        """
        Создает список задач и героев, которые могут их выполнять.
        :param tasks: Список задач
        :return: Список задач с возможными героями и ролями
        """
        
        task_list = []
        for task in tasks:
            possible_roles = []
            for hero in self.heroes_table:
                if hero['Запись в дневнике'] == task:
                    possible_roles.append((hero['Герой'], hero['Роль']))
            task_list.append((task, possible_roles))
        return task_list

    def calculate_team_scores(self, team, t0):
        """
        Рассчитать средние оценки команды и время выполнения задачи.
        :param team: Список героев и их ролей
        :param t0: Ожидаемое время выполнения задачи
        :return: Средние оценки и время выполнения
        """
        qualities = []
        deadlines = []
        politeness = []

        for hero, role in team:
            for record in self.heroes_table:
                if record['Герой'] == hero and record['Роль'] == role:
                    qualities.append(record['Оценка за качество'])
                    deadlines.append(record['Оценка по срокам'])
                    politeness.append(record['Оценка за вежливость'])

        avg_quality = sum(qualities) / len(qualities) if qualities else 0
        avg_deadline = sum(deadlines) / len(deadlines) if deadlines else 0
        max_politeness = max(politeness) if politeness else 0

        
        task_time = math.ceil(t0 * 5 / avg_deadline) if avg_deadline > 0 else float('inf')

        
        weighted_score = (
            0.5 * avg_quality +
            0.4 * avg_deadline +
            0.1 * max_politeness
        )

        return avg_quality, avg_deadline, max_politeness, task_time, weighted_score

    def check_team_availability(self, team, start_quant, task_duration):
        """
        Проверяет, есть ли пересечение доступности команды в нужном интервале времени.
        :param team: Список героев и их ролей
        :param start_quant: Квант времени начала задачи
        :param task_duration: Длительность задачи
        :return: Tuple[bool, Set[int]]: True, если команда доступна, и множество пересечений
        """
        team_availability = []

        for hero, _ in team:
            hero_availability = self.hero_availability.get(hero, [])
            team_availability.append(set(hero_availability))

        common_availability = set.intersection(*team_availability) if team_availability else set()

        for start in common_availability:
            required_time = list(range(start, start + task_duration))
            if all(quant in common_availability for quant in required_time):
                if required_time[0] >= start_quant:
                    return True, common_availability

        return False, common_availability

    def generate_teams(self, task_list, start_quant, t0, max_team_size=4):
        """
        Генерация всех возможных команд для выполнения задач.
        :param task_list: Список задач с возможными героями
        :param start_quant: Квант времени начала задачи
        :param t0: Ожидаемое время выполнения задачи
        :param max_team_size: Максимальный размер команды
        :return: Список возможных команд
        """
        teams = []

        def backtrack(current_team, task_index, used_roles):
            if task_index == len(task_list):
                if 1 <= len(current_team) <= max_team_size:
                    avg_quality, avg_deadline, max_politeness, task_duration, weighted_score = self.calculate_team_scores(current_team, t0)

                    is_available, common_availability = self.check_team_availability(current_team, start_quant, task_duration)
                    if is_available:
                        teams.append({
                            'team': list(current_team),
                            'avg_quality': avg_quality,
                            'avg_deadline': avg_deadline,
                            'max_politeness': max_politeness,
                            'task_time': task_duration,
                            'weighted_score': weighted_score,
                            'common_availability': common_availability,
                        })
                return

            task, candidates = task_list[task_index]
            for hero, role in candidates:
                if (hero, role) not in used_roles:
                    current_team.append((hero, role))
                    used_roles.add((hero, role))
                    backtrack(current_team, task_index + 1, used_roles)
                    used_roles.remove((hero, role))
                    current_team.pop()

        backtrack([], 0, set())
        return teams



### Пример работы алгоритм по созданию команд

In [ ]:
# Пример входных данных
heroes_table = [
    {'Герой': 'Агата', 'Роль': 'лучник', 'Оценка за качество': 4, 'Оценка по срокам': 4, 'Оценка за вежливость': 4, 'Запись в дневнике': 'разжечь костёр'},
    {'Герой': 'Агата', 'Роль': 'рейнджер', 'Оценка за качество': 4, 'Оценка по срокам': 4, 'Оценка за вежливость': 4, 'Запись в дневнике': 'разжечь костёр'},
    {'Герой': 'Борис', 'Роль': 'целитель', 'Оценка за качество': 5, 'Оценка по срокам': 5, 'Оценка за вежливость': 5, 'Запись в дневнике': 'залечить раны'},
]

heroes_table = hero_profiles[hero_profiles["cluster"]=="прогнать-проучить разбойник"].T.to_dict().values()
tasks = ['разжечь костёр', 'разжечь костёр', 'залечить раны']

# Начальные параметры задачи
start_quant = 1100
t0 = 3


# Создание и использование TaskManager
manager = TaskManager(heroes_table, heroes_availibility)
task_mapping = manager.find_heroes_for_task(tasks)
teams = manager.generate_teams(task_mapping, start_quant, t0)
sorted_teams = sorted(teams, key=lambda team: team['weighted_score'], reverse=True)

for team_info in teams:
    print("Команда:", team_info['team'])
    print("Средняя оценка за качество:", team_info['avg_quality'])
    print("Средняя оценка по срокам:", team_info['avg_deadline'])
    print("Максимальная оценка за вежливость:", team_info['max_politeness'])
    print("Время выполнения задачи:", team_info['task_time'])
    print("---")


Все возможные команды и их оценки:
Команда: [('Агата', 'лучник'), ('Агата', 'рейнджер'), ('Альфред', 'лекарь')]
Средняя оценка за качество: 4.0
Средняя оценка по срокам: 3.777777777777778
Максимальная оценка за вежливость: 4.666666666666667
Время выполнения задачи: 4
---
Команда: [('Агата', 'лучник'), ('Агата', 'рейнджер'), ('Мартин', 'лекарь')]
Средняя оценка за качество: 4.0
Средняя оценка по срокам: 3.777777777777778
Максимальная оценка за вежливость: 4.666666666666667
Время выполнения задачи: 4
---
Команда: [('Агата', 'лучник'), ('Агата', 'рейнджер'), ('Соня', 'лекарь')]
Средняя оценка за качество: 3.8333333333333335
Средняя оценка по срокам: 3.9444444444444446
Максимальная оценка за вежливость: 4.666666666666667
Время выполнения задачи: 4
---
Команда: [('Агата', 'лучник'), ('Агата', 'рейнджер'), ('Фредерик', 'лекарь')]
Средняя оценка за качество: 4.0
Средняя оценка по срокам: 3.777777777777778
Максимальная оценка за вежливость: 4.666666666666667
Время выполнения задачи: 4
---
Кома

### Метод для формирования словаря задач

Был создан метод, который на выходе возвращает словарь следующего формата:

```python
{
    task: {
        "priority": priorities,
        "start_quant": start_quant,
        "amount": amount,
        "nearest": nearest
    }
}

```
----

### Описание структуры словаря:

* **priority**: Список допустимых команд, отсортированных по убыванию их приоритетов для выполнения задачи.
* **start_quant**: Квант времени, соответствующий дате поступления задачи.
* **amount**: Стоимость выполнения задачи.
* **nearest**: Ближайшая задача, которая связана с текущей задачей.

---

### Зачем это нужно?
Данный метод упрощает управление задачами и позволяет:

* Оптимально распределять задачи между командами с учётом их приоритета.
* Учитывать временные факторы и взаимосвязи задач.
* Подготовить данные для дальнейшей интеграции в рекомендательную систему.

Далее будет описано, как эти данные используются для формирования оптимальных рекомендаций и управления задачами

In [139]:
def priority_wrapper(task, sorted_teams, start_quant, amount, nearest):
    priorities = {}
    for id, team in enumerate(sorted_teams):
        priorities[id] = team
   
    return {task : {
        "priority" : priorities,
        "start_quant" : start_quant,
        "amount": amount,
        "nearest": nearest
        }
    }

### Анализ исторических данных и создание векторного представления кластеров

Для работы с историческими данными из обучающей выборки был разработан вспомогательный метод ``vectorize_by_clusters``, который:

1. Позволяет получить информацию по каждому кластеру, включая:
   - Список поручений, относящихся к кластеру.
   - Их векторное представление для дальнейшего анализа.

---

#### Использование модели трансформера

Для преобразования текстового описания поручений в векторное представление была использована модель **"ruselkomp/deep-pavlov-full"**. Эта модель, обученная на русском языке, выполняет задачи обработки естественного языка и обеспечивает качественное векторизирование текстов.

---

#### Особенности векторизации

1. **Исключение из описания:**  
   Перед векторизацией из описания поручений удаляются:
   - **Совет:** Дополнительные рекомендации, которые не влияют на суть задачи.
   - **Цель:** Общая информация о задаче, которая уже классифицирована в кластерах.

2. **Результат:**  
   Каждое поручение преобразуется в числовой вектор, представляющий его основные семантические особенности.

---

#### Пример работы метода

- **Входное описание поручения:**  
  "В деревне был похищен рюкзак. Необходимо вернуть его хозяину. Советуем искать в лесу."

- **Преобразованное описание:**  
  "В деревне был похищен рюкзак."

- **Векторное представление:**  
  Вектор из чисел, который передаётся в алгоритмы для анализа и кластеризации.

---

#### Зачем это нужно?

Создание векторного представления задач:
1. **Упрощает анализ:** Позволяет сравнивать задачи внутри одного кластера.
2. **Поддерживает обучение модели:** Использование векторов упрощает внедрение алгоритмов машинного обучения.
3. **Повышает точность классификации:** Векторизация фокусируется на ключевых аспектах описания задачи.

Данный метод создаёт основу для дальнейшего анализа и оптимизации распределения задач.


In [ ]:
model_name = "ruselkomp/deep-pavlov-full"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def encode_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # [CLS] токен как представление предложения
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.squeeze().numpy()

Some weights of BertModel were not initialized from the model checkpoint at ruselkomp/deep-pavlov-full and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [141]:
def vectorize_by_clusters(dt_train_cases):
    clusters = {}
    for cluster in df_train_cases["cluster"].unique():
        df_slice = df_train_cases[df_train_cases["cluster"] == cluster].copy()
        vecs_slice = encode_text(df_slice["description"].values.tolist())
        
        clusters[cluster] = {
            "df_slice":
                df_train_cases[df_train_cases["cluster"] == cluster],
            "vecs": vecs_slice
            }
    return clusters

In [142]:
clusters = vectorize_by_clusters(df_train_cases)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [143]:
clusters

{'убить-победить-уничтожить зверь': {'df_slice':      Номер поручения      Заказчик Дата поручения Дата выполнения  \
  0              11000          Анна     1051-08-11      1051-08-21   
  5              11005          Анна     1051-09-07      1051-09-24   
  10             11010          Анна     1051-02-12      1051-02-18   
  14             11014       Татьяна     1052-05-16      1052-05-30   
  24             11024        Эмилио     1051-06-04      1051-07-21   
  50             11050         Чарли     1052-09-01      1052-09-16   
  80             11081       Татьяна     1052-05-29      1052-06-27   
  128            11129          Олег     1051-04-23      1051-05-03   
  135            11137  Бабушка Синь     1051-02-21      1051-03-03   
  156            11159          Егор     1051-12-24      1051-12-31   
  201            11205       Татьяна     1053-07-22      1053-08-13   
  205            11209       Леонтия     1052-04-20      1052-05-23   
  215            11220        

# 3. Модель

### Определение ближайших выполненных поручений перед запуском модели

Перед запуском модели для планирования невыполненных поручений был реализован подход, который позволяет использовать информацию из исторических данных выполненных задач.

---

#### Алгоритм определения ближайшего выполненного поручения

1. **Определение кластера для невыполненного поручения**  
   Для каждого невыполненного поручения определяется его **кластер цель-объект**, чтобы ограничить поиск задач только задачами из того же кластера.

2. **Поиск ближайшего выполненного поручения**  
   Используется векторное представление поручений. Для входного невыполненного поручения рассчитывается **косинусное расстояние** до всех выполненных поручений внутри этого же кластера.  
   - Находится ближайшая задача, которая уже была успешно выполнена.

3. **Получение данных о выполненной задаче**  
   Для ближайшего выполненного поручения извлекается:
   - **Идеальное время выполнения задачи.**
   - **Список задач, необходимых для выполнения.**

---

#### Допущение

Предполагается, что все задачи внутри одного кластера схожи локально. Это позволяет использовать данные о ближайших выполненных задачах как ориентир для оценки времени выполнения и необходимых действий для текущего невыполненного поручения.

---

#### Построение команд для невыполненной задачи

1. **Формирование допустимых команд**  
   На основе списка задач ближайшего выполненного поручения рассчитываются допустимые команды (с помощью ``TaskManager``), которые могут выполнить аналогичные действия.

2. **Оценка команд**  
   Для каждой команды рассчитываются:
   - Средние показатели по качеству, срокам и вежливости.
   - Ожидаемое время выполнения задачи.

---

#### Приоритизация допустимых команд

На основе полученных данных для каждой невыполненной задачи формируется список **приоритетных допустимых команд**, которые:
1. Имеют наивысшую **средневзвешенную оценку**:
   
   * **Оценка команды = 0.5 * Качество + 0.4 * Сроки + 0.1 * Вежливость**
   
2. Учитывают доступность героев в заданном временном интервале.

---

### Итог

Этот подход позволяет:
- Использовать данные выполненных задач для предсказания времени и стратегии выполнения новых поручений.
- Формировать приоритетные команды, которые оптимальны с точки зрения эффективности и доступности.
- Подготовить базу для реализации рекомендательной системы с учётом успешного опыта выполнения схожих задач.


### Формирование тестовой выборки

In [144]:
df_test_cases = df_cases[df_cases["Выполнено"]=="нет"]
df_test_cases.head()

,Номер поручения,Заказчик,Дата поручения,Выполнено,Дата выполнения,Затрачено дней,Сумма вознаграждения,Описание,description,advice,purpose,purpose_prep,description_prep,common_purpose,description_object,cluster,time_in,time_finish,Дата поручения_t,Дата выполнения_t
56,11056,Мария,1053-09-06,нет,NaN,NaN,23500,По дороге из деревни у меня пропала драгоценно...,По дороге из деревни у меня пропала драгоценность,,Нужно вернуть её как можно скорее.,вернуть,дорога деревня пропасть драгоценность,вернуть-найти,драгоценность,вернуть-найти драгоценность,1053-09-06 00:00:00,NaN,978,-1
134,11134,Иван,1053-09-04,нет,NaN,NaN,27500,В пещере появвилось огромное каменное чудовище...,В пещере появвилось огромное каменное чудовище,,Нужно уничтожить его.,уничтожить,пещера появвилось огромный каменный чудовище,убить-победить-уничтожить,огромный каменный чудовище,убить-победить-уничтожить огромный каменный чу...,1053-09-04 00:00:00,NaN,976,-1
143,11143,Егор,1053-10-18,нет,NaN,NaN,10500,В пещере завёлся дракон. Нужно его убить. Это ...,В пещере завёлся дракон,"Это может потребовать времени и усилий, так ка...",Нужно его убить,убить,пещера завестись дракон,убить-победить-уничтожить,дракон,убить-победить-уничтожить дракон,1053-10-18 00:00:00,NaN,1020,-1
161,11161,Эмилио,1053-10-03,нет,NaN,NaN,7500,В городе у меня потерялся рюкзак. Нужно найти ...,В городе у меня потерялся рюкзак,,Нужно найти его как можно скорее.,найти,город потеряться рюкзак,вернуть-найти,рюкзак,вернуть-найти рюкзак,1053-10-03 00:00:00,NaN,1005,-1
218,11218,Олег,1053-10-01,нет,NaN,NaN,16000,Недалеко от города монстры похитили путников. ...,Недалеко от города монстры похитили путников,"Осмотрите все возможные укрытия, чтобы найти п...",Нужно спасти их,спасти,город монстр похитить путник,спасти-освободить,монстр,спасти-освободить монстр,1053-10-01 00:00:00,NaN,1003,-1


In [ ]:
#Нахождение ближайшего выполненного соседа и создание на основе его данных всевозмодных команд
wrappers = []
tasks_teams = {}
for task_name, clust, case_, start_quant in df_test_cases[["Номер поручения", "cluster","description", "Дата поручения_t"]].values:
    df_slice, vecs = clusters[clust].values()
    input_vector = encode_text([case_])
    similarities = cosine_similarity([input_vector], vecs).flatten()
    most_similar_idx = similarities.argmax()
    most_similar_record = df_slice.iloc[most_similar_idx]  
    
    heroes_table = hero_profiles[hero_profiles["cluster"]==clust].T.to_dict().values()
    most_similar_record[["Сумма вознаграждения"]]

    nearest_task, t0, amount, tasks = most_similar_record[["Номер поручения", "Идеальное время", "Сумма вознаграждения", "Запись в дневнике"]].values


    manager = TaskManager(heroes_table, heroes_availibility)
    task_mapping = manager.find_heroes_for_task(tasks)
    teams = manager.generate_teams(task_mapping, start_quant, t0)
    sorted_teams = sorted(teams, key=lambda team: team['weighted_score'], reverse=True)

    wrappers.append(priority_wrapper(task_name, sorted_teams, start_quant, amount, nearest_task))


for d in wrappers:
    tasks_teams.update(d)

### Пример полученной конструкции

In [146]:
tasks_teams[11056]

{'priority': {0: {'team': [('Синеглазый', 'следопыт'), ('Соня', 'следопыт')],
   'avg_quality': 4.416666666666666,
   'avg_deadline': 4.0,
   'max_politeness': 3.75,
   'task_time': 8,
   'weighted_score': 4.183333333333334,
   'common_availability': {1,
    1088,
    1089,
    1090,
    1091,
    1092,
    1093,
    1094,
    1095,
    1096,
    1097,
    1098,
    1099,
    1100,
    1101,
    1102,
    1103,
    1104,
    1105,
    1106}},
  1: {'team': [('Соня', 'следопыт'), ('Синеглазый', 'следопыт')],
   'avg_quality': 4.416666666666666,
   'avg_deadline': 4.0,
   'max_politeness': 3.75,
   'task_time': 8,
   'weighted_score': 4.183333333333334,
   'common_availability': {1,
    1088,
    1089,
    1090,
    1091,
    1092,
    1093,
    1094,
    1095,
    1096,
    1097,
    1098,
    1099,
    1100,
    1101,
    1102,
    1103,
    1104,
    1105,
    1106}},
  2: {'team': [('Мартин', 'следопыт'), ('Синеглазый', 'следопыт')],
   'avg_quality': 4.125,
   'avg_deadline': 4.0,
 

### Обратный расчет приоритетов команд по задачам

После того как для каждой задачи были рассчитаны допустимые команды и их приоритеты, выполнена обратная задача:  

**Определение приоритетов задач для каждой команды.**

---

#### Цель обратного расчета

Для каждой команды необходимо:
1. Рассчитать список задач, которые эта команда может выполнить.
2. Определить приоритетность задач, основанную на **стоимости поручения**:
   - **Более дорогие задачи получают более высокий приоритет.**
   - Это позволяет оптимально распределить ресурсы команды на более ценные задачи.

---

#### Логика расчета

1. **Список доступных задач**  
   Для каждой команды формируется список задач, которые она может выполнить, с учётом:
   - Доступности команды в заданном временном интервале.
   - Возможности выполнения всех подзадач поручения.

2. **Сортировка по стоимости**  
   Задачи в списке команды сортируются по **убыванию стоимости**:
   - Более дорогие задачи имеют больший приоритет для выполнения.
   

---

Допущение:
- Стоимость задачи не учитывает её сложность и идеальное время выполнения.

---

3. **Результат:**  
   Для каждой команды создаётся список задач с приоритетами.

---

#### Зачем это нужно?

1. **Оптимизация распределения ресурсов**  
   Позволяет направить команды на выполнение наиболее ценных задач, увеличивая общую эффективность системы.

2. **Балансирование нагрузки**  
   Учитывает доступность команд и позволяет равномерно распределить задачи между ними.

3. **Подготовка к построению финального расписания**  
   Этот этап закладывает основу для эффективного выполнения задач с учётом приоритетов как задач, так и команд.

---

### Итог

Расчёт приоритетов задач для каждой команды завершает процесс подготовки данных для построения рекомендательной системы. Теперь команды могут быть распределены на задачи с учётом их стоимости и временной доступности, что обеспечивает максимальную эффективность выполнения поручений.


In [147]:
team_priorities = {}
for task_id, task_data in tasks_teams.items():
    amount = task_data["amount"]
    for team_info in task_data["priority"].values():
        team = tuple(team_info["team"])
        if team not in team_priorities:
            team_priorities[team] = [(task_id, amount)]
        else:
            team_priorities[team].append((task_id, amount))

team_priorities_sorted = {}
for key, value in team_priorities.items():
    team_priorities_sorted[key] = dict([[id, values] for id, values in enumerate(sorted(value, reverse=True, key=lambda x: x[-1]))])

# Постановка задачи: Модель двухсторонних рынков с временными отрезками в виде задачи целочисленного программирования

**Аркаша**, зная классические методы построения рекомендательных систем, решил поэкспериментировать и искать оптимальные команды для выполнения задач, используя подходы из теории двухсторонних рынков. Он сформулировал задачу **распределения задач между командами** как модель двухстороннего рынка с временными отрезками, где важно учитывать доступность команд в гильдии, чтобы успешно выполнить все поручения.

## Что такое задача двухсторонних рынков?

Двухсторонний рынок — это модель, где два множества агентов (например, работники и работодатели) взаимодействуют, чтобы сформировать "сочетания", удовлетворяющие их предпочтениям. Классическим примером является задача распределения интернов по больницам, где:
- Больницы имеют ограниченное количество мест и список предпочтений интернов.
- Интерны предпочитают работать в определённых больницах.

В рамках таких задач часто вводится понятие **блокирующей пары**:
- **Блокирующая пара** — это пара (агент из первого множества, агент из второго множества), которая предпочитает друг друга своим текущим партнёрам, нарушая стабильность системы.

Аркаша адаптировал эту идею для распределения задач между командами, где:
1. Задачи ("работодатели") имеют приоритеты среди команд.
2. Команды ("работники") имеют приоритеты среди задач.
3. Важно учитывать не только стабильность, но и временные ограничения.

---

## Основные множества

### 1. Задачи (tasks)
Это множество задач, каждая из которых имеет следующие параметры:
- `start_quant`: минимальный квант времени, когда задача может быть начата.
- `amount`: вознаграждение за успешное выполнение задачи.
- `priority`: словарь, где каждому приоритету (ключу) соответствует описание, включающее:
  - Название команды (`team`) — список пар кортежей вида `[(Член команды, Роль)]`.
  - Время выполнения задачи данной командой (`task_time`).
  - Доступные кванты времени для всей команды (`common_availability`).

### 2. Команды (teams)
Это множество команд, где каждая команда представлена как:
- Список пар кортежей вида `[(Член команды, Роль)]`.
- Каждая команда имеет список задач с приоритетами, которые она может выполнять.

---

## Переменные

### 1. Бинарная переменная $X_{\text{team, task, t}}$
- $X_{\text{team, task, t}} = 1$ , если команда `team` начинает выполнение задачи `task` в квант времени \( t \).
- Эта переменная нужна для:
  - Связи задачи с командой и временем её выполнения.
  - Учёта доступности команд в гильдии в конкретные временные интервалы.

### 2. Бинарная переменная $w_{\text{task}}$
- $w_{\text{task}} = 1$, если задача `task` выполнена.
- Эта переменная нужна для:
  - Отслеживания, была ли задача выполнена.
  - Подсчёта выполненных задач в целевой функции.

### 3. Бинарная переменная $B_{\text{team, task}}$
- $B_{\text{team, task}} = 1$, если команда `team` и задача `task` образуют блокирующую пару.
- Эта переменная нужна для:
  - Учёта стабильности решения.
  - Сокращения числа блокирующих пар.

### 4. Вещественная неотрицательная переменная $\text{MaxTime}$
- Это максимальное время выполнения задач.
- Нужна для:
  - Минимизации общего времени завершения всех задач.

---

## Ограничения

### 1. Связь выполнения задач с командами и временем
$$
w_{\text{task}} = \sum_{\text{team}, t} X_{\text{team, task, t}}
$$
- Это ограничение устанавливает, что задача считается выполненной, если она была начата хотя бы одной командой в определённый момент времени.
- **Зачем нужно:** чтобы задача могла считаться выполненной только при наличии команды, которая действительно начала её выполнение.

---

### 2. Ограничение времени
Если $X_{\text{team, task, t}} = 1$, то все команды, в которых есть участники команды `team`, становятся недоступны на время выполнения задачи `task` (в интервале $[t, t + \text{task\_time}]$).
$$
\sum_{\text{team1, task1, t1}} X_{\text{team1, task1, t1}} \leq M \cdot (1 - X_{\text{team, task, t}})
$$
- **Зачем нужно:** чтобы гарантировать, что одна и та же команда или её члены не могут одновременно выполнять разные задачи.

---

### 3. Ограничение слабой стабильности
$$
q_h X_{\text{team, task, t}} + q_h \sum_{j \succ \text{team}} X_{j, \text{task}, t} + \sum_{i \succ \text{task}} X_{\text{team}, i, t} \geq q_h - M \cdot B_{\text{team, task}}
$$
- **Зачем нужно:** чтобы уменьшить количество блокирующих пар, при которых команды и задачи предпочитают друг друга больше, чем текущим партнёрам.
- $q_{\text{h}}$ - количество команд, которая может принять задача. Будем считать $q_{\text{h}}=1$
- ${\text{M}}$ - большое число, которое отключает ограничение, при разных значения бинарных переменных
---

### 4. Максимальное время выполнения
$$
\text{MaxTime} \geq t + \text{task\_time} \cdot X_{\text{team, task, t}}
$$
- **Зачем нужно:** чтобы минимизировать время завершения самой поздней задачи.

---

### 5. Минимизация отклонения от предпочтительного времени начала задачи
$$
\sum_{\text{team, task, t}} \left(t - \text{start\_quant}_{\text{task}}\right) \cdot X_{\text{team, task, t}}
$$
- **Зачем нужно:** чтобы задачи выполнялись ближе к предпочтительному времени их начала.

---

## Целевая функция

Итоговая целевая функция объединяет несколько компонентов:
$$
\text{Objective} = -\text{TotalReward} + \text{MaxTime} + \alpha_1 \cdot \sum (1 - w_{\text{task}}) + \alpha_2 \cdot \sum B_{\text{team, task}} + \beta \cdot \sum_{\text{team, task, t}} \left(t - \text{start\_quant}_{\text{task}}\right) $$

Где:
- $ \text{TotalReward} $: сумма вознаграждений за выполненные задачи.
- $ \text{MaxTime} $: максимальное время выполнения задач.
- $ \alpha_1 $: штраф за невыполнение задач.
- $ \alpha_2 $: штраф за блокирующие пары.
- $ \beta $: штраф за отклонение времени выполнения от $ \text{start\_quant} $.


---

## Задача

Аркаша построил модель двухстороннего рынка с временными отрезками, чтобы распределять задачи между командами так, чтобы:
1. Максимизировать сумму вознаграждений за задачи.
2. Минимизировать максимальное время выполнения задач.
3. Уменьшить количество невыполненных задач.
4. Уменьшить количество блокирующих пар.
5. Учитывать предпочтительное время выполнения задач.




In [ ]:
class TaskAssignmentProblem:
    def __init__(self, tasks, teams, planning_horizon, alpha_1=0, alpha_2=0, beta=1000):
        """
        Инициализация задачи распределения задач между командами.

        :param tasks: Словарь с данными задач.
        :param teams: Словарь с данными команд.
        :param planning_horizon: Горизонт планирования (список квантов времени).
        :param alpha_1: Весовой коэффициент для штрафа за невыполненные задачи.
        :param alpha_2: Весовой коэффициент для штрафа за блокирующие пары.
        :param beta: Весовой коэффициент для штрафа за отклонение времени выполнения.
        """
        self.tasks = tasks
        self.teams = teams
        self.T = planning_horizon
        self.alpha_1 = alpha_1
        self.alpha_2 = alpha_2
        self.alpha_3 = beta

        # Создание модели
        self.model = LpProblem(name="TaskAssignmentProblem", sense=LpMinimize)

        # Переменные
        self.X = {}
        self.w = {}
        self.B = {}
        self.max_time = None

        # Вызов методов
        self._create_variables()
        self._add_constraints()
        self._set_objective()

    def _create_variables(self):
        """Создание переменных модели."""
        for task_id, task_data in self.tasks.items():
            start_quant = task_data["start_quant"]
            for team in [tuple(priority["team"]) for priority in task_data["priority"].values()]:
                for t in range(start_quant, max(self.T) + 1):
                    self.X[team, task_id, t] = LpVariable(f"X_{team}_{task_id}_{t}", cat="Binary")

        for task_id in self.tasks.keys():
            self.w[task_id] = LpVariable(f"w_{task_id}", cat="Binary")

        for task_id, task_data in self.tasks.items():
            for team in [tuple(priority["team"]) for priority in task_data["priority"].values()]:
                self.B[team, task_id] = LpVariable(f"B_{team}_{task_id}", cat="Binary")

        self.max_time = LpVariable("MaxTime", lowBound=0, cat="Continuous")

    def _add_constraints(self):
        """Добавление ограничений в модель."""
        for task_id, task_data in self.tasks.items():
            start_quant = task_data["start_quant"]
            self.model += (
                self.w[task_id]
                == lpSum(
                    self.X[team, task_id, t]
                    for team in [tuple(priority["team"]) for priority in task_data["priority"].values()]
                    for t in range(start_quant, max(self.T) + 1)
                ),
                f"TaskCompletion_{task_id}",
            )

        for task_id, task_data in self.tasks.items():
            start_quant = task_data["start_quant"]
            for team in [tuple(priority["team"]) for priority in task_data["priority"].values()]:
                task_time = next(
                    priority["task_time"]
                    for priority in task_data["priority"].values()
                    if tuple(priority["team"]) == team
                )
                for t in range(start_quant, max(self.T) + 1 - task_time):
                    self.model += (
                        self.max_time
                        >= t + task_time * self.X[team, task_id, t],
                        f"MaxTime_{team}_{task_id}_{t}",
                    )

        M = 1000000
        for task_id, task_data in self.tasks.items():
            for team in [tuple(priority["team"]) for priority in task_data["priority"].values()]:
                start_quant = task_data["start_quant"]
                task_time = next(
                    priority["task_time"]
                    for priority in task_data["priority"].values()
                    if tuple(priority["team"]) == team
                )

                self.model += (
                    lpSum(
                        self.X[team, task_id, t]
                        for t in range(start_quant, max(self.T) + 1 - task_time)
                    )
                    >= 1 - M * self.B[team, task_id],
                    f"WeakStability_{team}_{task_id}",
                )

    def _set_objective(self):
        """Задание целевой функции."""
        total_reward = lpSum(
            self.w[task_id] * self.tasks[task_id]["amount"]
            for task_id in self.tasks.keys()
        )
        uncompleted_tasks_penalty = lpSum(1 - self.w[task_id] for task_id in self.tasks.keys())
        blocking_pairs_penalty = lpSum(
            self.B[team, task_id]
            for task_id in self.tasks.keys()
            for team in [tuple(priority["team"]) for priority in self.tasks[task_id]["priority"].values()]
        )
        deviation_penalty = lpSum(
            (t - self.tasks[task_id]["start_quant"]) * self.X[team, task_id, t]
            for task_id in self.tasks.keys()
            for team in [tuple(priority["team"]) for priority in self.tasks[task_id]["priority"].values()]
            for t in range(self.tasks[task_id]["start_quant"], max(self.T) + 1)
        )

        self.model += (
            -total_reward
            + self.alpha_3 * self.max_time
            + self.alpha_1 * uncompleted_tasks_penalty
            + self.alpha_2 * blocking_pairs_penalty
            + self.alpha_3 * deviation_penalty,
            "ObjectiveFunction",
        )

    def solve(self):
        """Решение задачи."""
        self.model.solve()
        return {
            "status": self.model.status,
            "objective": self.model.objective.value(),
            "variables": {v.name: v.value() for v in self.model.variables()},
        }


In [244]:
problem = TaskAssignmentProblem(tasks=tasks_teams, teams=team_priorities_sorted, planning_horizon=t, alpha_1=1, alpha_2=1, beta=1)

In [245]:
result = problem.solve()

In [ ]:
def parse_variable_name(variable_name):
    """
    Разбирает имя переменной X_(team)_task_t и извлекает команду, задачу и время.
    
    :param variable_name: Название переменной (строка).
    :return: tuple (team, task, start_time).
    """
    match = re.match(r"X_\((.+)\)_(\d+)_(\d+)", variable_name)
    if not match:
        return

    team_str, task, start_time = match.groups()
    team_str = team_str.replace("_", " ")

    try:
        team = eval(team_str)
    except SyntaxError as e:
        return
    return team, int(task), int(start_time)


In [247]:
def calculate_total_reward(tasks, w):
    """
    Вычисляет значение TotalReward из решения модели PuLP.

    :param tasks: Словарь задач, содержащий информацию о вознаграждении ('amount').
    :param w: Словарь переменных w_task из решения модели (0 или 1).
    :return: Значение TotalReward.
    """
    total_reward = 0
    for task_id, task_data in tasks.items():
        if w[f"w_{task_id}"] == 1:  # Если задача выполнена
            total_reward += task_data["amount"]
    return total_reward

Достанем результаты модели:

In [248]:
variables = result.get("variables", {})
w_values = {name: value for name, value in variables.items() if name.startswith("w_")}
x_values = {name: value for name, value in variables.items() if name.startswith("X_")}
maxTime =  {name: value for name, value in variables.items() if name.startswith("MaxTime")}


### Выполнены все задачи

In [249]:
sum(x_values.values()) == len(tasks_teams)

True

### Суммарная выручка

In [250]:
total_reward = calculate_total_reward(tasks_teams, w_values)
total_reward

343500

In [ ]:
#Подсчет основных характеристик назначенных команд на поручения
tasks_completed = []
for x, value in x_values.items():
    if value > 0:
        team, task_id, start_time = parse_variable_name(x)
        for pripority, team_info in tasks_teams[task_id]["priority"].items():
            if team_info["team"] == list(team):
                tasks_completed.append(
                    {
                        "Номер поручения": task_id,
                        "Команда": team,
                        "Затрачено дней": team_info["task_time"],
                        "Время начала выполнения": start_time,
                        "Время завершения": team_info["task_time"] + start_time,
                        "Оценка за качество": team_info["avg_quality"],
                        "Оценка по срокам": team_info["avg_deadline"],
                        "Оценка за вежливость": team_info["max_politeness"],
                        "Приоритет команды": pripority,

                    }
                )
        

### Создание финальной таблицы распределения команд по поручениям

In [253]:
result_df_from_model = pd.DataFrame(tasks_completed)
result_df_from_model.head()

,Номер поручения,Команда,Затрачено дней,Время начала выполнения,Время завершения,Оценка за качество,Оценка по срокам,Оценка за вежливость,Приоритет команды
0,11143,"((Альфред, лекарь), (Альфред, мечник))",6,1020,1026,3.833333,3.666667,3.500000,51
1,11234,"((Бендер, мечник), (Мартин, лучник))",14,1026,1040,3.833333,3.416667,4.500000,418
2,11396,"((Бендер, рейнджер), (Юлия, рейнджер))",3,1033,1036,4.166667,3.833333,4.333333,81
3,11402,"((Бендер, следопыт), (Фредерик, следопыт))",3,1045,1048,3.628571,4.057143,4.000000,50
4,11232,"((Бенедикт, следопыт),)",5,1024,1029,3.600000,4.200000,4.200000,4


In [254]:
result_df_from_model["Дата начала выполнения"] = result_df_from_model["Время начала выполнения"].map(t_date_dict)
result_df_from_model["Дата завершения"] = result_df_from_model["Время завершения"].map(t_date_dict)

In [263]:
result_df = pd.merge(
    result_df_from_model,
    df_test_cases[["Номер поручения", "Дата поручения", "Дата поручения_t", "Описание", "cluster"]],
    on="Номер поручения",
)

### Запись распределения Аркашей героев по невыполненным задачам в result_task_assignment.csv

In [264]:
result_df

,Номер поручения,Команда,Затрачено дней,Время начала выполнения,Время завершения,Оценка за качество,Оценка по срокам,Оценка за вежливость,Приоритет команды,Дата начала выполнения,Дата завершения,Дата поручения,Дата поручения_t,Описание,cluster
0,11143,"((Альфред, лекарь), (Альфред, мечник))",6,1020,1026,3.833333,3.666667,3.500000,51,1053-10-18 00:00:00,1053-10-24 00:00:00,1053-10-18,1020,В пещере завёлся дракон. Нужно его убить. Это ...,убить-победить-уничтожить дракон
1,11234,"((Бендер, мечник), (Мартин, лучник))",14,1026,1040,3.833333,3.416667,4.500000,418,1053-10-24 00:00:00,1053-11-07 00:00:00,1053-10-24,1026,В лесу по дороге от пещеры заметили разбойнико...,прогнать-проучить разбойник
2,11396,"((Бендер, рейнджер), (Юлия, рейнджер))",3,1033,1036,4.166667,3.833333,4.333333,81,1053-10-31 00:00:00,1053-11-03 00:00:00,1053-10-31,1033,В лесу недалеко от города заметили зверей. Нуж...,убить-победить-уничтожить зверь
3,11402,"((Бендер, следопыт), (Фредерик, следопыт))",3,1045,1048,3.628571,4.057143,4.000000,50,1053-11-12 00:00:00,1053-11-15 00:00:00,1053-11-12,1045,По дороге из деревни у меня потерялась драгоце...,вернуть-найти драгоценность
4,11232,"((Бенедикт, следопыт),)",5,1024,1029,3.600000,4.200000,4.200000,4,1053-10-22 00:00:00,1053-10-27 00:00:00,1053-10-22,1024,В деревне монстры похитили путников. Нужно спа...,спасти-освободить монстр
5,11306,"((Бенедикт, следопыт),)",5,1048,1053,3.600000,4.200000,4.200000,4,1053-11-15 00:00:00,1053-11-20 00:00:00,1053-11-15,1048,В деревне монстры похитили путников. Нужно спа...,спасти-освободить монстр
6,11462,"((Бенедикт, следопыт),)",4,1055,1059,3.600000,4.200000,4.200000,4,1053-11-22 00:00:00,1053-11-26 00:00:00,1053-11-22,1055,В городе монстры похитили путников. Нужно осво...,спасти-освободить монстр
7,11417,"((Леопольд, следопыт), (Пастушок, следопыт))",12,994,1006,3.904762,3.452381,3.833333,80,1053-09-22 00:00:00,1053-10-04 00:00:00,1053-09-22,994,Недалеко от города у меня была украдена драгоц...,вернуть-найти драгоценность
8,11387,"((Леопольд, следопыт), (Юлия, следопыт))",6,1015,1021,4.083333,2.916667,4.000000,100,1053-10-13 00:00:00,1053-10-19 00:00:00,1053-10-13,1015,Недалеко от города у меня пропала драгоценност...,вернуть-найти драгоценность
9,11218,"((Мартин, следопыт),)",4,1003,1007,4.000000,4.000000,4.500000,1,1053-10-01 00:00:00,1053-10-05 00:00:00,1053-10-01,1003,Недалеко от города монстры похитили путников. ...,спасти-освободить монстр


In [257]:
result_df.to_csv("result_task_assignment.csv")

### Аркаша успешно составил и распределил команды так, что все невыполненные задачи были выполнены

#### Среднее время ожидания начала выполнения задачи в днях представлено ниже:
#### Можно видеть, что модель стремилась минимизировать время ожидания, поэтому были подобраны такие команды, которые готовы были взяться за задачи сразу же, как они поступили

In [267]:
np.mean(result_df["Время начала выполнения"] -result_df["Дата поручения_t"])

0.0

#### Суммарное время выполнения (в днях):

In [269]:
np.mean(result_df["Затрачено дней"])

6.2105263157894735

## Проверка случайного распределения задач

Для проверки алгоритма случайного распределения задач мы реализуем следующий подход:

### Основные предположения:
1. **Набор команд и их приоритеты**:
   - Используем `team_priorities_sorted` — множество, где каждая команда ассоциирована с приоритетами на выполнение задач.
   - Если у команды есть приоритет на задачу, то она может её выполнять.

2. **Индексация задач**:
   - Все задачи, на которые необходимо назначить команды, индексированы от $0$ до $n$, где $n$ — общее количество задач, которые остались невыполненными.

### Процесс распределения:
1. **Случайный выбор команд**:
   - Из множества команд ``team_priorities_sorted`` выбираем случайный набор команд. Это позволяет учитывать различные сочетания назначений.

2. **Назначение задач**:
   - Для каждой задачи $i$ проверяем, доступна ли случайно выбранная команда $\text{team}_i$ (под индексом $i$) для выполнения задачи $ i $ в соответствии с приоритетами.
   - Если команда может выполнить задачу, то она назначается на неё.

### Упрощения и ограничения:
1. **Игнорирование доступности команд**:
   - Фактор доступности команд (например, ограниченность ресурсов или уже назначенные задачи) не учитывается.
   - Это объясняется тем, что текущее распределение изначально не покрывает все задачи, а цель проверки — оценить случайное распределение.

2. **Игнорирование временного фактора**:
   - Временные ограничения (например, доступные кванты времени для выполнения задач) также не учитываются.
   - Учет временного фактора требует отдельной постановки задачи, время на которую не хватило в рамках поставленного дедлайна.

### Замечание:
Случайное распределение предназначено для грубой оценки и проверки корректности базового подхода к назначению задач. Оно не стремится к оптимальности и не учитывает ключевые ограничения, такие как доступность. 

Реализация полного алгоритма с учетом временных ограничений требует дополнительного времени на разработку, которое в условиях контеста отсутствует.


In [ ]:
#Случайные выбор n команд
def get_teams_distribution(feasible_teams, num_teams, random_seed=42):
    random.seed(random_seed)
    return dict(random.sample(list(feasible_teams.items()), num_teams))

In [ ]:
#Невыполненные задачи
required_tasks_ids = df_test_cases["Номер поручения"].values
required_tasks_ids

array([11056, 11134, 11143, 11161, 11218, 11232, 11234, 11285, 11306,
       11310, 11311, 11381, 11387, 11396, 11402, 11417, 11428, 11438,
       11462], dtype=int64)

In [ ]:
#Проверка, подходит команда к  поручению
def check_team_competence(team, task_id, tasks):
    task = tasks[task_id]["priority"]
    for team_info in task.values():
        if team_info["team"] == list(team):
            return {
                "Номер поручения": task_id,
                "Команда": team,
                "Оценка за качество": team_info["avg_quality"],
                "Оценка по срокам": team_info["avg_deadline"],
                "Оценка за вежливость": team_info["max_politeness"],
                "Затрачено дней": team_info["task_time"],
                "Награда": tasks[task_id]["amount"]  
            }
    return -1

In [ ]:
#Отбор случайных команд из списка, которые смогли бы выполнить задачу без учета времени
def get_random_team_completed_tasks(teams, required_tasks_ids, tasks_info):
    result = []
    for id_, team in enumerate(teams):
        task_id = required_tasks_ids[id_]
        competence = check_team_competence(team, task_id, tasks_info)
        if competence != -1:
            result.append(competence)
    return pd.DataFrame(result)

## Как можно видеть, случайное распределение команд не позволяет выполнить все еще невыполненные поручения гильдии даже с учетом отсутсвия фактора времени

In [329]:
random_teams_42 = get_teams_distribution(team_priorities_sorted, len(required_tasks_ids), 42)
df_res_teams_42 = get_random_team_completed_tasks(random_teams_42, required_tasks_ids, tasks_teams)
df_res_teams_42

,Номер поручения,Команда,Оценка за качество,Оценка по срокам,Оценка за вежливость,Затрачено дней,Награда
0,11387,"((Соня, следопыт), (Юлия, следопыт))",4.416667,3.250000,4.000000,5,29000
1,11428,"((Юлия, мечник), (Альфред, мечник))",3.333333,3.833333,4.333333,7,15000


In [330]:
df_res_teams_42["Награда"].sum()

44000

In [332]:
random_teams_24 = get_teams_distribution(team_priorities_sorted, len(required_tasks_ids), 24)
df_res_teams_24 = get_random_team_completed_tasks(random_teams_24, required_tasks_ids, tasks_teams)
df_res_teams_24

,Номер поручения,Команда,Оценка за качество,Оценка по срокам,Оценка за вежливость,Затрачено дней,Награда
0,11285,"((Альфред, следопыт), (Пастушок, следопыт))",3.916667,3.750000,4.333333,3,16000
1,11311,"((Фредерик, следопыт), (Синеглазый, следопыт))",4.178571,3.857143,3.857143,7,22000
2,11402,"((Мартин, следопыт), (Пастушок, следопыт))",3.946429,3.785714,4.250000,3,15000


In [333]:
df_res_teams_24["Награда"].sum()

53000

In [334]:
random_teams_35 = get_teams_distribution(team_priorities_sorted, len(required_tasks_ids), 35)
df_res_teams_35 = get_random_team_completed_tasks(random_teams_35, required_tasks_ids, tasks_teams)
df_res_teams_35

,Номер поручения,Команда,Оценка за качество,Оценка по срокам,Оценка за вежливость,Затрачено дней,Награда
0,11143,"((Мартин, лекарь), (Бендер, мечник))",4.321429,4.125000,4.142857,5,11000
1,11234,"((Бенедикт, рейнджер), (Синеглазый, рейнджер))",4.000000,3.875000,4.500000,12,26000
2,11402,"((Альфред, следопыт), (Агата, следопыт))",3.416667,3.666667,4.500000,3,15000


In [335]:
df_res_teams_24["Награда"].sum()

53000

# Мотивация и особенности модели

Модель была разработана с целью минимизации времени ожидания клиентов. Это ключевой фактор для повышения лояльности заказчиков: чем быстрее гильдия выполняет заказы, тем выше вероятность того, что клиенты вернутся снова. Именно поэтому  модели выбирала такие команды, чье максимальное число героев в командах было ограничено двумя. Этот выбор объясняется тем, что именно команды из двух героев позволили учесть графики всех участников, синхронизировать их доступность и начинать выполнение заданий как можно раньше.

## Условия и допущения

1. **Ограничения на качество работы героев**:
   - Каждый герой может быть выбран для выполнения задачи только в том случае, если его средняя оценка за качество работы превышает 3.
   - Также учитывается пунктуальность: герой допускается к работе, если его средние оценки за выполнение в срок превышают 2.
   - Эти ограничения позволяют поддерживать высокий уровень качества выполнения заданий.

2. **Гибкость параметров приоритетов**:
   - Пользователь модели может настроить параметры $ \alpha_1$, $\alpha_2$,$\beta$, чтобы управлять акцентами при оптимизации:
     - $\alpha_1 $: важность количества выполнения заданий.
     - $\alpha_2$: важность соблюдения приоритетов задач.
     - $ \beta$: важность отклонения от начала выполнения задачи с момента ее поступления.
   - Это даёт возможность адаптировать модель под различные сценарии — от строгого соблюдения сроков до выполнения большего числа заказов.

## Итоги работы модели

Данный подход оказался наиболее эффективным. Он обеспечил:
- Учет графиков каждого героя, что позволило начинать задачи в максимально ранние сроки.
- Соответствие высокого уровня качества, заданного через средние оценки.
- Возможность пользователям модели выбирать, что для них важнее: минимизация времени, соблюдение приоритетов или увеличение числа выполненных задач.

Такая гибкость делает модель подходящей как для малых гильдий с небольшим числом заказов, так и для крупных организаций, где важно учитывать множество различных факторов при планировании работы.


# Результаты работы

В результате выполнения работы были достигнуты следующие этапы и результаты:

1. **Анализ описаний поручений:**
   - Описания поручений были разделены на **описание**, **цель**, и **совет**.
   - Это позволило структурировать данные и выделить ключевую информацию.

2. **Определение объектов задания:**
   - С помощью моделей трансформеров из текстового описания был извлечён **объект задания**.
   - Это улучшило понимание контекста задачи и её связи с целью.

3. **Кластеризация задач:**
   - Построены **кластеры цель-объект**, что позволило определить основные типы задач в гильдии.
   - Это упростило группировку и анализ поручений.

4. **Анализ ценообразования:**
   - Было проанализировано ценообразование с точки зрения заказчиков.
   - Принято решение **не учитывать фактор заказчика** при формировании рекомендаций, так как он не влияет на эффективность выполнения задач.

5. **Анализ дневников героев:**
   - Изучены дневники героев, их выполненные задачи и оценки.
   - Оценки героев были агрегированы по каждому показателю (качество, сроки, вежливость) внутри их ролей и кластеров.

6. **Фильтрация героев:**
   - Герои были признаны **недопустимыми** для выполнения задач в определённых ролях и кластерах, если:
     - Средняя оценка за качество < 3.
     - Средняя оценка за сроки ≤ 2.

7. **Интервалы доступности:**
   - Для каждого героя определены **интервалы доступности** между поручениями.

---

### Построение рекомендательной системы

1. **Определение ближайших соседей задач:**
   - Для каждой невыполненной задачи найден её **ближайший выполненный сосед** с точки зрения векторного описания (модель трансформера) и общего кластера.

2. **Использование информации о соседях:**
   - Из ближайших выполненных задач извлечены:
     - **Идеальное время выполнения**, рассчитанное по формуле.
     - **Список действий**, необходимых для выполнения задачи.

3. **Формирование команд:**
   - На основе кластера и доступных героев были построены команды, которые:
     - Закрывают все подзадачи для выполнения поручения.
     - Имеют свободное временное окно в момент появления задачи.

4. **Оценка команд:**
   - Рассчитаны средние оценки команды за:
     - **Качество.**
     - **Сроки.**
     - **Максимальная вежливость** (представляет всю команду).
   - Средняя оценка команды рассчитывалась как:
     $
     \text{Оценка команды} = 0.5 \times \text{Качество} + 0.4 \times \text{Сроки} + 0.1 \times \text{Вежливость}
     $

5. **Приоритеты команд:**
   - Задачи отсортированы по приоритетам, основываясь на оценке команды.
   - Для всех команд определены приоритеты по стоимости задач (команды стремились к выполнению самых дорогих поручений).

---

### Оптимизация с помощью целочисленного программирования

1. На основе множества задач и команд сформулирована задача **целочисленного программирования**, учитывающая:
   - Принципы двухсторонних рынков.
   - Временной фактор выполнения поручений.

2. **Расширяемость модели:**
   - Подход позволяет варьировать гиперпараметры в зависимости от приоритетов пользователя (например, важность времени, приоритетности, % невыполненных задач).

3. **Результаты модели:**
   - Модель способна:
     - Назначать команды на задачи с учётом времени и эффективности.
     - Избегать временных пересечений и противоречий.
     - Обеспечивать максимальную ценность выполнения задач для гильдии.

---

### Итог

Разработанный подход даёт гибкость и точность в распределении задач. Он позволяет учитывать множество факторов, таких как время, качество, стоимость и доступность героев, обеспечивая оптимальное выполнение поручений.


# Допущения, сделанные при разработке модели

1. **Сложность задач**  
   - Сложность задач явно не определялась. Главными критериями для оптимизации были:
     - **Стоимость задачи.**
     - **Доступность героев.**
   - Это могло упростить некоторые аспекты планирования.

2. **Ограничение выполнения задач героями**  
   - Герой считается способным выполнять задачу в заданном кластере и роли, если:
     - Его средняя оценка за сроки > 2.
     - Его средняя оценка за качество > 3.  
   - Это ограничение основано на усреднённых данных и не учитывает индивидуальные случаи.

3. **Интерпретация оценки по срокам**  
   - Оценка по срокам интерпретировалась как **время выполнения задачи в днях**.  
   - Это упрощение могло игнорировать более тонкие аспекты временного фактора.

4. **Идеальное время выполнения**  
   - Идеальное время выполнения задачи рассчитывалось по заданной формуле, а не предсказывалось на основе более сложных моделей.  
   - Это могло снизить точность оценки времени для задач, сильно отличающихся от типичных.

5. **Слабая интерпретация вежливости**  
   - Оценка по вежливости учитывалась как максимальное значение в команде, что может быть недостаточно точным.  
   - Более детальный подход мог бы улучшить понимание влияния вежливости на результаты.

6. **Отсутствие анализа оценок с точки зрения заказчика**  
   - Оценки задач не подвергались более глубокому анализу с точки зрения:
     - **Требований заказчиков.**
     - **Причин их удовлетворённости или неудовлетворённости.**

7. **Частичное игнорирование времени выполнения подзадач**  
   - Время, затраченное героями на выполнение конкретных подзадач, не учитывалось.
   - Учитывание этих данных могло бы улучшить точность оценки сложности и эффективности выполнения задач.

---

### Итог

Допущения позволили упростить разработку модели и сосредоточиться на ключевых аспектах. Однако дальнейшее расширение этих ограничений могло бы сделать модель более точной и адаптированной к реальным условиям. Это остаётся перспективой для будущей работы.
